In [49]:
import pandas as pd
%load_ext sql
%sql sqlite:///sqlite-sakila.db
veriler = r"sqlite-sakila.db"

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [50]:
import sqlite3

conn = sqlite3.connect(veriler)

cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

for table in tables:
    print(table[0])

actor
country
city
address
language
category
customer
film
film_actor
film_category
film_text
inventory
staff
store
payment
rental


-----------------------------------
1. TÜM VERİLERİ GÖSTER
-----------------------------------

In [51]:
conn = sqlite3.connect(veriler)

df = pd.read_sql_query("SELECT * FROM rental LIMIT 10;", conn)

print(df.head())

   rental_id              rental_date  inventory_id  customer_id  \
0          1  2005-05-24 22:53:30.000           367          130   
1          2  2005-05-24 22:54:33.000          1525          459   
2          3  2005-05-24 23:03:39.000          1711          408   
3          4  2005-05-24 23:04:41.000          2452          333   
4          5  2005-05-24 23:05:21.000          2079          222   

               return_date  staff_id          last_update  
0  2005-05-26 22:04:30.000         1  2021-03-06 15:53:41  
1  2005-05-28 19:40:33.000         1  2021-03-06 15:53:41  
2  2005-06-01 22:12:39.000         1  2021-03-06 15:53:41  
3  2005-06-03 01:43:41.000         2  2021-03-06 15:53:41  
4  2005-06-02 04:33:21.000         1  2021-03-06 15:53:41  


-----------------------------------
2. HER FİLMDEKİ OYUNCULARI LİSTELE
-----------------------------------

In [52]:
df = {}

for table_name in ["film", "actor", "film_actor"]:
    query = f"SELECT * FROM {table_name}"
    cursor = conn.execute(query)
    cols = [col[0] for col in cursor.description]
    df[table_name] = pd.DataFrame(cursor.fetchall(), columns=cols)

listOfActor = (
    df["film"].merge(df["film_actor"], on="film_id", how="inner").merge(df["actor"], on="actor_id", how="inner")
)

sonuc = listOfActor[["title", "first_name", "last_name"]]

print(sonuc)

                  title first_name  last_name
0      ACADEMY DINOSAUR   PENELOPE    GUINESS
1      ACADEMY DINOSAUR  CHRISTIAN      GABLE
2      ACADEMY DINOSAUR    LUCILLE      TRACY
3      ACADEMY DINOSAUR     SANDRA       PECK
4      ACADEMY DINOSAUR     JOHNNY       CAGE
...                 ...        ...        ...
5457  ZOOLANDER FICTION     WHOOPI       HURT
5458  ZOOLANDER FICTION       JADA      RYDER
5459          ZORRO ARK        IAN      TANDY
5460          ZORRO ARK       NICK  DEGENERES
5461          ZORRO ARK       LISA     MONROE

[5462 rows x 3 columns]


-----------------------------------
3. HER FİLMDE KAÇ OYUNCU OYNADI
-----------------------------------

In [53]:
df = {}

for table_name in ["film", "actor", "film_actor"]:
    query = f"SELECT * FROM {table_name}"
    cursor = conn.execute(query)
    cols = [col[0] for col in cursor.description]
    df[table_name] = pd.DataFrame(cursor.fetchall(), columns=cols)

listOfActor = (
    df["film"].merge(df["film_actor"], on="film_id", how="inner").merge(df["actor"], on="actor_id", how="inner")
)

Actor = listOfActor.groupby("title").size().reset_index(name="actor_count")

sonuc = Actor

print(sonuc)

                 title  actor_count
0     ACADEMY DINOSAUR           10
1       ACE GOLDFINGER            4
2     ADAPTATION HOLES            5
3     AFFAIR PREJUDICE            5
4          AFRICAN EGG            5
..                 ...          ...
992     YOUNG LANGUAGE            5
993         YOUTH KICK            5
994       ZHIVAGO CORE            6
995  ZOOLANDER FICTION            5
996          ZORRO ARK            3

[997 rows x 2 columns]


-----------------------------------
4. HER OYUNCU KAÇ FİLMDE OYNADI?
-----------------------------------

In [54]:
df = {}

for table_name in ["film", "actor", "film_actor"]:
    query = f"SELECT * FROM {table_name}"
    cursor = conn.execute(query)
    cols = [col[0] for col in cursor.description]
    df[table_name] = pd.DataFrame(cursor.fetchall(), columns=cols)

actor_movie = (
    df["actor"].merge(df["film_actor"], on = "actor_id", how="inner").merge(df["film"], on = "film_id", how="inner")
)

TotalCount = actor_movie.groupby(["first_name", "last_name"]).size().reset_index(name = "movie_count")

sonuc = TotalCount

print(sonuc)

    first_name  last_name  movie_count
0         ADAM      GRANT           18
1         ADAM     HOPPER           22
2           AL    GARLAND           26
3         ALAN   DREYFUSS           27
4       ALBERT  JOHANSSON           33
..         ...        ...          ...
194       WILL     WILSON           31
195    WILLIAM    HACKMAN           27
196      WOODY    HOFFMAN           31
197      WOODY      JOLIE           31
198       ZERO       CAGE           25

[199 rows x 3 columns]


-----------------------------------
5. ENVANTERDE OLMAYAN FİLMLER VAR MI VE VARSA KAÇ TANE?
-----------------------------------

In [58]:
df = {}

for table_name in ["film", "inventory"]:
    query = f"SELECT * FROM {table_name}"
    cursor = conn.execute(query)
    cols = [col[0] for col in cursor.description]
    df[table_name] = pd.DataFrame(cursor.fetchall(), columns=cols)

movie_review = (
    df['film'][~df['film']['film_id'].isin(df['inventory']['film_id'])].shape[0]
)

sonuc = pd.DataFrame({"movie_review": [movie_review]})

print(sonuc)

   movie_review
0            42


-----------------------------------
6. KİRALANABİLİR OLAN HER FİLMİN KAÇ KEZ KİRALANDIĞINI VE TOPLAM GELİRLERİNİ GETİRİN
-----------------------------------

In [72]:
df = {}
for table_name in ["film", "inventory", "rental", "payment"]:
    query = f"SELECT * FROM {table_name}"
    cursor = conn.execute(query)
    cols = [col[0] for col in cursor.description]
    df[table_name] = pd.DataFrame(cursor.fetchall(), columns=cols)

rent = (
    df["film"]
      .merge(df["inventory"], on="film_id", how="left", suffixes=("", "_inv"))
      .merge(df["rental"], on="inventory_id", how="left", suffixes=("", "_rental"))
      .merge(df["payment"], on="rental_id", how="left", suffixes=("", "_payment"))
      .groupby("title", as_index=False)
      .agg(
          kiralama=("rental_id", "count"),
          total_payment=("amount", "sum")
      )
      .sort_values(["kiralama", "total_payment"], ascending=[False, False])
      .reset_index(drop=True)
)

print(rent.head())

                 title  kiralama  total_payment
0   BUCKET BROTHERHOOD        34         180.66
1     ROCKETEER MOTHER        33         116.67
2        SCALAWAG DUCK        32         172.68
3  RIDGEMONT SUBMARINE        32         130.68
4       FORWARD TEMPLE        32         128.68


-----------------------------------
7. ENVANTERDE OLMAYAN FİLMLERİN KİRA ORANLARINI GETİRİN
-----------------------------------

In [73]:
df = {}

for table_name in ["film", "inventory"]:
    query = f"SELECT * FROM {table_name}"
    cursor = conn.execute(query)
    cols = [col[0] for col in cursor.description]
    df[table_name] = pd.DataFrame(cursor.fetchall(), columns=cols)


rent_rate = df["film"][~df["film"]["film_id"].isin(df["inventory"]["film_id"])][["title", "rental_rate"]]

rent_rate.columns = ["fimler", "kiralama_oranları"]

print(rent_rate)

                     fimler  kiralama_oranları
13           ALICE FANTASIA               0.99
32              APOLLO TEEN               2.99
35           ARGONAUTS TOWN               0.99
37            ARK RIDGEMONT               0.99
40     ARSENIC INDEPENDENCE               0.99
86        BOONDOCK BALLROOM               0.99
107           BUTCH PANTHER               0.99
127           CATCH AMISTAD               0.99
143     CHINATOWN GLADIATOR               4.99
147          CHOCOLATE DUCK               2.99
170    COMMANDMENTS EXPRESS               4.99
191        CROSSING DIVORCE               4.99
194         CROWDS TELEMARK               4.99
197        CRYSTAL BREAKING               2.99
216              DAZED PUNK               4.99
220  DELIVERANCE MULHOLLAND               0.99
317       FIREHOUSE VIETNAM               0.99
324           FLOATS GARDEN               2.99
331   FRANKENSTEIN STRANGER               0.99
358      GLADIATOR WESTWARD               4.99
385          

-----------------------------------
8. BİRDEN FAZLA DVD'Yİ İADE ETMEYEN KAÇ MÜŞTERİ VAR?
-----------------------------------

In [77]:
df = {}

for table_name in ["rental"]:
    query = f"SELECT * FROM {table_name}"
    cursor = conn.execute(query)
    cols = [col[0] for col in cursor.description]
    df[table_name] = pd.DataFrame(cursor.fetchall(), columns=cols)

customer_count = (df["rental"].query("return_date.isnull()", engine="python").customer_id.nunique()
)

unreturn_customer_count = pd.DataFrame({"customer_count": [customer_count]})

print(unreturn_customer_count.head())

   customer_count
0             159


-----------------------------------
9. HER MÜŞTERİ KAÇ FİLM KİRALADI?
-----------------------------------

In [76]:
df = {}

for table_name in ["customer", "rental"]:
    query = f"SELECT * FROM {table_name}"
    cursor = conn.execute(query)
    cols = [col[0] for col in cursor.description]
    df[table_name] = pd.DataFrame(cursor.fetchall(), columns=cols)

rentals_per_customer = (
    df["customer"]
      .merge(df["rental"], on="customer_id", how="left")
      .groupby(["first_name", "last_name"], as_index=False)
      .agg(rental_count=("rental_id", "count"))
      .sort_values("rental_count", ascending=False)
    .reset_index(drop=True)
)

print(rentals_per_customer.head())

  first_name last_name  rental_count
0    ELEANOR      HUNT            46
1       KARL      SEAL            45
2     MARCIA      DEAN            42
3      CLARA      SHAW            42
4      TAMMY   SANDERS            41


-----------------------------------
10. TÜRLERİNE GÖRE EN ÇOK KİRALANAN FİLMLER VE BUNLARA NE KADAR ÖDENDİ?
-----------------------------------

In [79]:
df = {}

for table_name in ["customer", "rental"]:
    query = f"SELECT * FROM {table_name}"
    cursor = conn.execute(query)
    cols = [col[0] for col in cursor.description]
    df[table_name] = pd.DataFrame(cursor.fetchall(), columns=cols)

rent_count = (df["customer"]
    .merge(df["rental"], on="customer_id", how="left")
    .groupby(["first_name", "last_name"], as_index=False)
    .agg(rent_count=("rental_id", "count"))
    .reset_index(drop=True)
)

print(rent_count.head())

  first_name last_name  rent_count
0      AARON     SELBY          24
1       ADAM     GOOCH          22
2     ADRIAN     CLARY          19
3      AGNES    BISHOP          23
4       ALAN      KAHN          26


-----------------------------------
11. TÜR VE TARİHE GÖRE KİRALAMA SAYISI VE GELİR
-----------------------------------

In [80]:
df = {}

for table_name in ["film","film_category", "category","inventory", "rental", "payment"]:
    query = f"SELECT * FROM {table_name}"
    cursor = conn.execute(query)
    cols = [col[0] for col in cursor.description]
    df[table_name] = pd.DataFrame(cursor.fetchall(), columns=cols)

category_date_rent = (df["category"]
    .merge(df["film_category"], on="category_id", suffixes=("", "_cat")) \
    .merge(df["film"], on="film_id", suffixes=("", "_film")) \
    .merge(df["inventory"], on="film_id", suffixes=("", "_inv")) \
    .merge(df["rental"], on="inventory_id", suffixes=("", "_rental")) \
    .merge(df["payment"], on="rental_id", suffixes=("", "_payment")) \
    .assign(rental_date=lambda x: pd.to_datetime(x["rental_date"])) \
    .groupby(["name", "rental_date"]).agg(

        kiralanma_sayilari=("rental_id", "count"),
        toplam_gelirleri=("amount", "sum")

    ).reset_index().sort_values(by="rental_date")
)

print(category_date_rent.head())

           name         rental_date  kiralanma_sayilari  toplam_gelirleri
7142     Family 2005-05-24 22:53:30                   1              2.99
11039     Music 2005-05-24 22:54:33                   1              2.99
2241   Children 2005-05-24 23:03:39                   1              3.99
10205    Horror 2005-05-24 23:04:41                   1              4.99
2242   Children 2005-05-24 23:05:21                   1              6.99


-----------------------------------
12. KİRALANABİLİR FİLMLER İÇİN TÜRLERİNE GÖRE HER FİLMİN KAÇ KEZ KİRALANDIĞI
-----------------------------------

In [94]:
df = {}

for table_name in ["film","film_category", "category","inventory", "rental"]:
    query = f"SELECT * FROM {table_name}"
    cursor = conn.execute(query)
    cols = [col[0] for col in cursor.description]
    df[table_name] = pd.DataFrame(cursor.fetchall(), columns=cols)

movie_rent = (df["category"]
    .merge(df["film_category"], on="category_id", suffixes=("", "_cat"))
    .merge(df["film"], on="film_id", suffixes=("", "_film"))
    .merge(df["inventory"], on="film_id", suffixes=("", "_inv"))
    .merge(df["rental"], on="inventory_id", suffixes=("", "_rental"))
    .groupby(["name", "title"])
    .agg(rental_count=("rental_id", "count"))
    .reset_index()
)

print(movie_rent.head())

     name                title  rental_count
0  Action         AMADEUS HOLY            21
1  Action      AMERICAN CIRCUS            22
2  Action   ANTITRUST TOMATOES            10
3  Action  BAREFOOT MANCHURIAN            18
4  Action         BERETS AGENT            21


-----------------------------------
13. EN ÇOK RAFTA BEKLEYEN FİLMLER
-----------------------------------_

In [97]:
df = {}

for table_name in ["film", "inventory", "rental"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

df["rental"]["rental_date"] = pd.to_datetime(df["rental"]["rental_date"], errors="coerce")
df["rental"]["return_date"] = pd.to_datetime(df["rental"]["return_date"], errors="coerce")

active = df["rental"][df["rental"]["return_date"].isna()].copy()

now = pd.Timestamp.now()
active["days_on_shelf"] = (now - active["rental_date"]).dt.days

on_shelf = (
    df["film"]
      .merge(df["inventory"], on="film_id", how="inner")
      .merge(active, on="inventory_id", how="inner")
      [["title", "days_on_shelf"]]
      .sort_values("days_on_shelf", ascending=False)
      .reset_index()
)

print(on_shelf.head())

   index                title  days_on_shelf
0      0     ACADEMY DINOSAUR           7339
1    115      MUMMY CREATURES           7162
2    117         NONE SPIKING           7162
3    118  OPERATION OPERATION           7162
4    119    OPPOSITE NECKLACE           7162


-----------------------------------
14. GEÇ, ERKEN VE ZAMANINDA İADE EDİLEN KİRALANMIŞ FİLMLER
-----------------------------------

In [101]:
# dataframe listesi oluşturma
df = {}

# Tek cursor kullanarak daha temiz okuma
query = "SELECT * FROM rental"
cur = conn.execute(query)
cols = [c[0] for c in cur.description]
df["rental"] = pd.DataFrame(cur.fetchall(), columns=cols)

rental_data = df["rental"].copy()

rental_data["rental_date"] = pd.to_datetime(rental_data["rental_date"], errors="coerce")
rental_data["return_date"] = pd.to_datetime(rental_data["return_date"], errors="coerce")

rental_data["due_date"] = rental_data["rental_date"] + pd.Timedelta(days=7)

valid = rental_data["return_date"].notna()

late_return = (valid & (rental_data["return_date"] > rental_data["due_date"])).sum()
early_return = (valid & (rental_data["return_date"] < rental_data["due_date"])).sum()
ontime_return = (valid & (rental_data["return_date"] == rental_data["due_date"])).sum()

print(f"Gecikmeli iade: {late_return}")
print(f"Erken iade: {early_return}")
print(f"Zamanında iade: {ontime_return}")


Gecikmeli iade: 4494
Erken iade: 11364
Zamanında iade: 3


-----------------------------------
15. HANGİ MÜŞTERİ EN ÇOK DVD KİRALAMIŞ?
-----------------------------------

In [105]:
df = {}

for table_name in ["customer", "rental", "inventory"]:
    query = f"SELECT * FROM {table_name}"
    cursor = conn.execute(query)
    cols = [col[0] for col in cursor.description]
    df[table_name] = pd.DataFrame(cursor.fetchall(), columns=cols)

max_dvd_rent = (df["customer"]
    .merge(df["rental"], on="customer_id", how="left")
    .merge(df["inventory"], on="inventory_id")
    .groupby(["first_name", "last_name"], as_index=False)
    .agg(rental_count=("rental_id", "count"))
    .reset_index(drop=True)
    .sort_values(by="rental_count", ascending=False)
)

print(max_dvd_rent.head(1))

    first_name last_name  rental_count
175    ELEANOR      HUNT            46


-----------------------------------
16. EN POPÜLER FİLM KATEGORİSİ NEDİR?
-----------------------------------

In [115]:
df = {}

for table_name in ["category", "film_category", "film","inventory", "rental"]:
    query = f"SELECT * FROM {table_name}"
    cursor = conn.execute(query)
    cols = [col[0] for col in cursor.description]
    df[table_name] = pd.DataFrame(cursor.fetchall(), columns=cols)

populer_kategori = (df["category"]
    .merge(df["film_category"], on="category_id", how ="left", suffixes=("_category", "_film_category"))
    .merge(df["film"], on="film_id", how ="left", suffixes=("_film_category", "_film"))
    .merge(df["inventory"], on="film_id", how ="left", suffixes=("_film", "_inventory"))
    .merge(df["rental"], on="inventory_id", how ="left", suffixes=("_inventory", "_rental"))
    .groupby("name")
    .agg(rent_count=("rental_id", "count"))
    .reset_index()
    .sort_values(by="rent_count", ascending=False)
)

print(populer_kategori.head(1))

      name  rent_count
14  Sports        1179


-----------------------------------
17. HANGİ ÇALIŞAN EN ÇOK KİRALAMA İŞLEMİ GERÇEKLEŞTİRMİŞTİR?
-----------------------------------

In [121]:
df = {}

for table_name in ["staff", "rental"]:
    query = f"SELECT * FROM {table_name}"
    cursor = conn.execute(query)
    cols = [col[0] for col in cursor.description]
    df[table_name] = pd.DataFrame(cursor.fetchall(), columns=cols)

max_rent = (df["staff"]
    .merge(df["rental"], on="staff_id", how ="left")
    .groupby(["first_name", "last_name"])
    .agg(rent_count=("rental_id", "count"))
    .reset_index()
    .sort_values(by="rent_count", ascending=False)
)

print(max_rent.head(1))

  first_name last_name  rent_count
1       Mike   Hillyer        8040


-----------------------------------
18. EN ÇOK GELİRİ HANGİ FİLM GETİRMİŞ ?
-----------------------------------

In [124]:
df = {}

for table_name in ["film", "inventory", "rental", "payment"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

top_film = (df["film"]
      .merge(df["inventory"], on="film_id", how="inner", suffixes=("", "_inv"))
      .merge(df["rental"], on="inventory_id", how="inner", suffixes=("", "_rent"))
      .merge(df["payment"], on="rental_id", how="inner", suffixes=("", "_pay"))
      .groupby("title", as_index=False)
      .agg(total_revenue=("amount", "sum"))
      .sort_values("total_revenue", ascending=False)
)

print(top_film.head(1))

                title  total_revenue
841  TELEGRAPH VOYAGE         231.73


-----------------------------------
19. HER MÜŞTERİ İÇİN TOPLAM HARCAMA MİKTARINI BULUN
-----------------------------------

In [132]:
df = {}

for table_name in ["rental", "payment", "customer"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

spend_per_customer = (
    df["customer"]
      .merge(df["rental"],  on="customer_id")     # customer -> rental
      .merge(df["payment"], on="rental_id")     # rental   -> payment
      .groupby(["first_name", "last_name"], as_index=False)
      .agg(total_spent=("amount", "sum"))
      .fillna({"total_spent": 0})
      .sort_values("total_spent", ascending=False)
      .reset_index(drop=True)
)

print(spend_per_customer.head(10))

  first_name last_name  total_spent
0       KARL      SEAL       221.55
1    ELEANOR      HUNT       216.54
2      CLARA      SHAW       195.58
3     RHONDA   KENNEDY       194.61
4     MARION    SNYDER       194.61
5      TOMMY   COLLAZO       186.62
6     WESLEY      BULL       177.60
7        TIM      CARY       175.61
8     MARCIA      DEAN       175.58
9        ANA   BRADLEY       174.66


-----------------------------------
20. HER KATEGORİDEKİ TOPLAM KİRALAMA SAYISINI VE GELİRLERİ BULUN
-----------------------------------

In [137]:
df = {}

for table_name in ["category", "film_category", "film", "inventory", "rental", "payment"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

category_stats = (
    df["category"]
      .merge(df["film_category"], on="category_id", suffixes=("_category", "_film_category"))
      .merge(df["film"],         on="film_id",      suffixes=("_film_category", "_film"))
      .merge(df["inventory"],    on="film_id",      suffixes=("_film", "_inventory"))
      .merge(df["rental"],       on="inventory_id", suffixes=("_inventory", "_rental"))
      .merge(df["payment"],      on="rental_id",    suffixes=("_rental", "_pay"))
      .groupby(["category_id", "name"], as_index=False)
      .agg(
          rental_count=("rental_id", "count"),
          total_revenue=("amount", "sum")
      )
      .sort_values(["rental_count", "total_revenue"], ascending=False)
      .reset_index(drop=True)
)

print(category_stats.head(10))

   category_id         name  rental_count  total_revenue
0           15       Sports          1179        5314.21
1            2    Animation          1166        4656.30
2            1       Action          1112        4375.85
3           14       Sci-Fi          1101        4756.98
4            8       Family          1096        4226.07
5            7        Drama          1060        4587.39
6            6  Documentary          1050        4217.52
7            9      Foreign          1033        4270.67
8           10        Games           969        4281.33
9            3     Children           945        3655.55


-----------------------------------
21. EN UZUN SÜRE KİRADA KALMIŞ FİLMLER
-----------------------------------

In [143]:
df = {}

for table_name in ["film", "inventory", "rental"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

# Kiralanma süresi tablosu
film_rental = (
    df["film"]
      .merge(df["inventory"], on="film_id", how="inner", suffixes=("", "_inv"))
      .merge(df["rental"], on="inventory_id", how="inner", suffixes=("", "_rent"))
)

# Sadece iade edilmişler
film_rental = film_rental[film_rental["return_date"].notnull()].copy()

# Tarih dönüşümü ve kiralama süresi (gün)
film_rental["rental_date"] = pd.to_datetime(film_rental["rental_date"], errors="coerce")
film_rental["return_date"] = pd.to_datetime(film_rental["return_date"], errors="coerce")
film_rental["rental_duration_days"] = (
    (film_rental["return_date"] - film_rental["rental_date"]).dt.total_seconds() / 86400
)

# Her film için maksimum kiralama süresi
film_rental_summary = (
    film_rental.groupby("title", as_index=False)
               .agg(max_rental_duration_days=("rental_duration_days", "max"))
               .sort_values("max_rental_duration_days", ascending=False)
               .head(10)
)

print(film_rental_summary)

                  title  max_rental_duration_days
400  HOLOCAUST HIGHBALL                  9.249306
393     HIGHBALL POTTER                  9.249306
160   CONEHEADS SMOOCHY                  9.248611
625          PANIC CLUB                  9.248611
535          MASK PEACH                  9.248611
868        TRAMP OTHERS                  9.248611
602   NOTORIOUS REUNION                  9.247222
38         ATTACKS HATE                  9.247222
863         TRACY CIDER                  9.246528
461        JERSEY SASSY                  9.246528


-----------------------------------
22. EN AZ KİRALANAN 5 FİLM HANGİLERİDİR?
-----------------------------------

In [144]:
df = {}

for table_name in ["film", "inventory", "rental"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

# En az kiralanan 5 film (kiralanmayanlar dahil)
least_rented = (
    df["film"]
      .merge(df["inventory"], on="film_id", how="left",  suffixes=("", "_inv"))
      .merge(df["rental"],   on="inventory_id", how="left", suffixes=("", "_rent"))
      .groupby("title", as_index=False)
      .agg(rental_count=("rental_id", "count"))
      .fillna({"rental_count": 0})
      .sort_values("rental_count", ascending=True, kind="stable")
      .head(5)
      .reset_index(drop=True)
)

print(least_rented)

                  title  rental_count
0        ALICE FANTASIA             0
1           APOLLO TEEN             0
2        ARGONAUTS TOWN             0
3         ARK RIDGEMONT             0
4  ARSENIC INDEPENDENCE             0


-----------------------------------
24. EN FAZLA KAZANÇ SAĞLAYAN 5 MÜŞTERİYİ BULUN
-----------------------------------

In [145]:
df = {}

for table_name in ["customer", "rental", "payment"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

max_earners = (
    df["customer"]
      .merge(df["rental"],  on="customer_id", how="left", suffixes=("", "_rent"))
      .merge(df["payment"], on="rental_id",   how="left", suffixes=("", "_pay"))
      .groupby(["customer_id", "first_name", "last_name"], as_index=False)
      .agg(total_spent=("amount", "sum"))
      .fillna({"total_spent": 0})
      .sort_values("total_spent", ascending=False)
      .head(5)
      .reset_index(drop=True)
)

print(max_earners)

   customer_id first_name last_name  total_spent
0          526       KARL      SEAL       221.55
1          148    ELEANOR      HUNT       216.54
2          144      CLARA      SHAW       195.58
3          137     RHONDA   KENNEDY       194.61
4          178     MARION    SNYDER       194.61


-----------------------------------
25. HER FİLMİN ORTALAMA KİRALANMA SÜRESİNİ BULUN
-----------------------------------

In [151]:
df = {}
for table_name in ["film", "inventory", "rental"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

merged_df = (
    df["film"]
      .merge(df["inventory"], on="film_id", how="inner", suffixes=("", "_inv"))
      .merge(df["rental"],   on="inventory_id", how="inner", suffixes=("", "_rent"))
      .copy()
)

merged_df = merged_df[merged_df["return_date"].notna()].copy()

merged_df["rental_date"] = pd.to_datetime(merged_df["rental_date"], errors="coerce")
merged_df["return_date"] = pd.to_datetime(merged_df["return_date"], errors="coerce")

merged_df["rental_duration_days"] = (
    (merged_df["return_date"] - merged_df["rental_date"]).dt.total_seconds() / 86400
)

avg_rental_duration = (
    merged_df.groupby("title", as_index=False)
             .agg(avg_rental_duration=("rental_duration_days", "mean"))
             .sort_values("avg_rental_duration", ascending=False)
             .reset_index(drop=True)
)

print(avg_rental_duration.head())

            title  avg_rental_duration
0     FLIGHT LIES             7.270775
1  IMPACT ALADDIN             7.200617
2     AFRICAN EGG             7.106629
3  HARDLY ROBBERS             6.948437
4    MADRE GABLES             6.811806


-----------------------------------
26. HER TÜRDE EN POPÜLER FİLMİ BULUN
-----------------------------------

In [150]:
df = {}

for table_name in ["category", "film_category", "film", "inventory", "rental"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

merged = (
    df["category"]
      .merge(df["film_category"], on="category_id", how="inner", suffixes=("", "_fc"))
      .merge(df["film"],          on="film_id",     how="inner", suffixes=("", "_film"))
      .merge(df["inventory"],     on="film_id",     how="inner", suffixes=("", "_inv"))
      .merge(df["rental"],        on="inventory_id",how="inner", suffixes=("", "_rent"))
)

popular_film = (
    merged.groupby(["name", "title"], as_index=False)
          .agg(rental_count=("rental_id", "count"))
          .rename(columns={"name": "genre"})
)

top_per_genre = (
    popular_film.loc[popular_film.groupby("genre")["rental_count"].idxmax()]
                .reset_index(drop=True)
)

print(top_per_genre)

          genre                title  rental_count
0        Action  RUGRATS SHAKESPEARE            30
1     Animation       JUGGLER HARDLY            32
2      Children         ROBBERS JOON            31
3      Classics       TIMBERLAND SKY            31
4        Comedy            ZORRO ARK            31
5   Documentary            WIFE TURN            31
6         Drama         HOBBIT ALIEN            31
7        Family        APACHE DIVINE            31
8       Foreign     ROCKETEER MOTHER            33
9         Games       FORWARD TEMPLE            32
10       Horror         PULP BEVERLY            30
11        Music        SCALAWAG DUCK            32
12          New  RIDGEMONT SUBMARINE            32
13       Sci-Fi    GOODFELLAS SALUTE            31
14       Sports  GLEAMING JAWBREAKER            29
15       Travel   BUCKET BROTHERHOOD            34


-----------------------------------
27. HER TÜRDE EN FAZLA GELİR SAĞLAYAN FİLMİ BULUN
-----------------------------------

In [152]:
df = {}

for table_name in ["category", "film_category", "film", "inventory", "rental", "payment"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

merged = (
    df["category"]
      .merge(df["film_category"], on="category_id", how="left",  suffixes=("", "_fc"))
      .merge(df["film"],          on="film_id",     how="left",  suffixes=("", "_film"))
      .merge(df["inventory"],     on="film_id",     how="left",  suffixes=("", "_inv"))
      .merge(df["rental"],        on="inventory_id",how="left",  suffixes=("", "_rent"))
      .merge(df["payment"],       on="rental_id",   how="left",  suffixes=("", "_pay"))
)

merged["amount"] = merged["amount"].fillna(0)
revenue_by_film = (
    merged.groupby(["name", "title"], as_index=False)
          .agg(total_revenue=("amount", "sum"))
          .rename(columns={"name": "genre"})
)

top_grossing_per_genre = (
    revenue_by_film.loc[revenue_by_film.groupby("genre")["total_revenue"].idxmax()]
                   .reset_index(drop=True)
)

print(top_grossing_per_genre)

          genre                title  total_revenue
0        Action     FOOL MOCKINGBIRD         175.77
1     Animation         DOGMA FAMILY         178.70
2      Children  BACKLASH UNDEFEATED         158.81
3      Classics          STEEL SANTA         141.77
4        Comedy            ZORRO ARK         214.69
5   Documentary            WIFE TURN         223.69
6         Drama         TORQUE BOUND         198.72
7        Family     RANGE MOONWALKER         179.73
8       Foreign       INNOCENT USUAL         191.74
9         Games       MASSACRE USUAL         179.70
10       Horror           LOLA AGENT         159.76
11        Music     TELEGRAPH VOYAGE         231.73
12          New          MAIDEN HOME         163.76
13       Sci-Fi    GOODFELLAS SALUTE         209.69
14       Sports       SATURDAY LAMBS         204.72
15       Travel   BUCKET BROTHERHOOD         180.66


-----------------------------------
28. EN ÇOK DVD İADE ETMEYEN MÜŞTERİYİ BULUN
-----------------------------------

In [154]:
df = {}

for table_name in ["customer", "rental"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

df["rental"]["return_date"] = pd.to_datetime(df["rental"]["return_date"], errors="coerce")

merged = (
    df["customer"]
      .merge(df["rental"], on="customer_id", how="left", suffixes=("", "_rent"))
)

merged["is_returned"] = merged["return_date"].notna()

min_returners = (
    merged.groupby(["first_name", "last_name"], as_index=False)
          .agg(returned_count=("is_returned", "sum"))   # True'lar toplanır
          .sort_values("returned_count", ascending=True)
          .head(1)
          .reset_index(drop=True)
)

print(min_returners)

  first_name last_name  returned_count
0      BRIAN     WYMAN              12


-----------------------------------
29. EN FAZLA KİRALAMA YAPAN 5 ÇALIŞANI BULUN
-----------------------------------

In [158]:
df = {}

for table_name in ["staff", "rental"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

max_rentals = (
    df["staff"]
      .merge(df["rental"], on="staff_id", how="left", suffixes=("", "_rent"))
      .groupby(["staff_id", "first_name", "last_name"], as_index=False)
      .agg(total_rentals=("rental_id", "count"))
      .sort_values("total_rentals", ascending=False)
      .head(5)
      .reset_index(drop=True)
)

print(max_rentals)

# en fazla 5 isteniyor ama 5 adet maksimum kiralama yapan yok, 2 adet olduğundan dolayı böyle bir çıktı veriyor.

   staff_id first_name last_name  total_rentals
0         1       Mike   Hillyer           8040
1         2        Jon  Stephens           8004


-----------------------------------
30. EN FAZLA KİRALAMA YAPAN 5 MÜŞTERİ HANGİ ŞUBEDEN KİRALAMA YAPMIŞ?
-----------------------------------

In [159]:
query = "PRAGMA table_info(store);"
columns = pd.read_sql_query(query, conn)

print(columns)

   cid              name       type  notnull dflt_value  pk
0    0          store_id        INT        1       None   1
1    1  manager_staff_id   SMALLINT        1       None   0
2    2        address_id        INT        1       None   0
3    3       last_update  TIMESTAMP        1       None   0


In [161]:
df = {}

for table_name in ["customer", "rental", "store", "address"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

top_renters = (
    df["customer"]
      .merge(df["rental"], on="customer_id", how="left", suffixes=("", "_rent"))
      .merge(df["store"],   on="store_id",   how="left", suffixes=("", "_store"))
      .groupby(["first_name", "last_name", "store_id"], as_index=False)
      .agg(total_rentals=("rental_id", "count"))
      .sort_values("total_rentals", ascending=False)
      .head(5)
      .reset_index(drop=True)
)

print(top_renters)

  first_name last_name  store_id  total_rentals
0    ELEANOR      HUNT         1             46
1       KARL      SEAL         2             45
2     MARCIA      DEAN         1             42
3      CLARA      SHAW         1             42
4      TAMMY   SANDERS         2             41


-----------------------------------
31. HER TÜRDE EN AZ KİRALANAN FİLMİ BULUN
-----------------------------------

In [171]:
df = {}
for table_name in ["category", "film_category", "film", "inventory", "rental"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

merged = (
    df["category"]
      .merge(df["film_category"], on="category_id", how="left", suffixes=("", "_fc"))
      .merge(df["film"],          on="film_id",     how="left", suffixes=("", "_film"))
      .merge(df["inventory"],     on="film_id",     how="left", suffixes=("", "_inv"))
      .merge(df["rental"],        on="inventory_id",how="left", suffixes=("", "_rent"))
)

popular = (
    merged.groupby(["name", "title"], as_index=False)
          .agg(rental_count=("rental_id", "count"))
)

least_rented = (
    popular.sort_values("rental_count", ascending=True, kind="stable")
           .head(5)
           .rename(columns={
               "name": "Category",
               "title": "Name",
               "rental_count": "Rental Count",
           })
)

print("Her Türde En Az Kiralanan Filmler:")
print(least_rented.to_string(index=False))

Her Türde En Az Kiralanan Filmler:
 Category              Name  Rental Count
   Action     ARK RIDGEMONT             0
   Action FIREHOUSE VIETNAM             0
   Action       SKY MIRACLE             0
Animation    ARGONAUTS TOWN             0
Animation     FLOATS GARDEN             0


-----------------------------------
32. EN ÇOK KİRALAMA YAPAN 5 MÜŞTERİ HANGİ ŞEHİRDE?
-----------------------------------

In [172]:
df = {}

for table_name in ["customer", "address", "city", "rental"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

top_customers = (
    df["customer"]
      .merge(df["address"], on="address_id", how="left", suffixes=("", "_addr"))
      .merge(df["city"],    on="city_id",    how="left", suffixes=("", "_city"))
      .merge(df["rental"],  on="customer_id",how="left", suffixes=("", "_rent"))
      .groupby(["city", "first_name", "last_name"], as_index=False)
      .agg(total_rentals=("rental_id", "count"))
      .sort_values("total_rentals", ascending=False)
      .head(5)
      .reset_index(drop=True)
      .rename(columns={
          "city": "city",
          "first_name": "first_name",
          "last_name": "last_name",
          "total_rentals": "total_rentals"
      })
)

print("Top 5 Customers by Rentals:")
print(top_customers.to_string(index=False))

Top 5 Customers by Rentals:
       city first_name last_name  total_rentals
Saint-Denis    ELEANOR      HUNT             46
 Cape Coral       KARL      SEAL             45
  Molodetno      CLARA      SHAW             42
      Tanza     MARCIA      DEAN             42
   Changhwa      TAMMY   SANDERS             41


-----------------------------------
33. EN ÇOK KAZANÇ SAĞLAYAN 5 MÜŞTERİYİ HANGİ ŞEHİRDE BULUN
-----------------------------------

In [173]:
df = {}

for table_name in ["customer", "address", "city", "rental", "payment"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

top_spenders = (
    df["customer"]
      .merge(df["address"], on="address_id", how="left", suffixes=("", "_addr"))
      .merge(df["city"],    on="city_id",    how="left", suffixes=("", "_city"))
      .merge(df["rental"],  on="customer_id",how="left", suffixes=("", "_rent"))
      .merge(df["payment"], on="rental_id",  how="left", suffixes=("", "_pay"))
      .groupby(["city", "first_name", "last_name"], as_index=False)
      .agg(total_spent=("amount", "sum"))
      .fillna({"total_spent": 0})
      .sort_values("total_spent", ascending=False)
      .head(5)
      .reset_index(drop=True)
)

print("Top 5 customers by total spending (with city):")
print(top_spenders[["city", "first_name", "last_name", "total_spent"]].to_string(index=False))

Top 5 customers by total spending (with city):
               city first_name last_name  total_spent
         Cape Coral       KARL      SEAL       221.55
        Saint-Denis    ELEANOR      HUNT       216.54
          Molodetno      CLARA      SHAW       195.58
          Apeldoorn     RHONDA   KENNEDY       194.61
Santa Brbara dOeste     MARION    SNYDER       194.61


-----------------------------------
34. EN ÇOK KİRALANAN 5 FİLM HANGİ ŞEHİRDE BULUN
-----------------------------------

In [174]:
df = {}

for table_name in ["city", "address", "customer", "rental", "inventory", "film"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

max_rented_cities = (
    df["city"]
      .merge(df["address"],  on="city_id",      how="left",  suffixes=("", "_addr"))
      .merge(df["customer"], on="address_id",   how="left",  suffixes=("", "_cust"))
      .merge(df["rental"],   on="customer_id",  how="left",  suffixes=("", "_rent"))
      .merge(df["inventory"],on="inventory_id", how="left",  suffixes=("", "_inv"))
      .merge(df["film"],     on="film_id",      how="left",  suffixes=("", "_film"))
      .groupby(["city", "title"], as_index=False)
      .agg(rental_count=("rental_id", "count"))
      .sort_values("rental_count", ascending=False)
      .head(7)
      .reset_index(drop=True)
)

print(max_rented_cities)

       city                title  rental_count
0   Trshavn    FLATLINERS KILLER             3
1  Sorocaba      CADDYSHACK JEDI             3
2    Kurgan     DETECTIVE VISION             3
3      Lima      DISCIPLE MOTHER             3
4   Caracas     AFFAIR PREJUDICE             2
5    Bhopal      GENTLEMEN STAGE             2
6    Bhopal  CANDIDATE PERDITION             2


-----------------------------------
35. EN AZ KİRALANAN 5 FİLM HANGİ ŞEHİRDE BULUN
-----------------------------------

In [175]:
df = {}

for table_name in ["city", "address", "customer", "rental", "inventory", "film"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

min_rented_cities = (
    df["city"]
      .merge(df["address"],  on="city_id",      how="left", suffixes=("", "_addr"))
      .merge(df["customer"], on="address_id",   how="left", suffixes=("", "_cust"))
      .merge(df["rental"],   on="customer_id",  how="left", suffixes=("", "_rent"))
      .merge(df["inventory"],on="inventory_id", how="left", suffixes=("", "_inv"))
      .merge(df["film"],     on="film_id",      how="left", suffixes=("", "_film"))
      .groupby(["city", "title"], as_index=False)
      .agg(kiralama_sayisi=("rental_id", "count"))
      .sort_values("kiralama_sayisi", ascending=True, kind="stable")
      .head(5)
      .rename(columns={
          "city": "şehri",
          "title": "filmin_adı",
          "kiralama_sayisi": "kiralanma_sayısı"  # çıktı etiketini isteğine göre Türkçe yaptım
      })
      .reset_index(drop=True)
)

print(min_kiralananlarin_sehirleri.to_string(index=False))

             şehri           filmin_adı  kiralanma_sayısı
A Corua (La Corua)         BLADE POLISH                 1
A Corua (La Corua)        CAPER MOTIONS                 1
A Corua (La Corua) CHICKEN HELLFIGHTERS                 1
A Corua (La Corua)  CHRISTMAS MOONSHINE                 1
A Corua (La Corua)         CIDER DESIRE                 1


-----------------------------------
36. EN ÇOK KAZANÇ SAĞLAYAN 5 FİLM HANGİ ŞEHİRDE BULUN
-----------------------------------

In [ ]:
df = {}

for table_name in ["city", "address", "customer", "rental", "inventory", "film", "payment"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

max_revenue_films_by_city = (
    df["city"]
      .merge(df["address"],  on="city_id",      how="inner", suffixes=("", "_addr"))
      .merge(df["customer"], on="address_id",   how="inner", suffixes=("", "_cust"))
      .merge(df["rental"],   on="customer_id",  how="inner", suffixes=("", "_rent"))
      .merge(df["inventory"],on="inventory_id", how="inner", suffixes=("", "_inv"))
      .merge(df["film"],     on="film_id",      how="inner", suffixes=("", "_film"))
      .merge(df["payment"],  on="rental_id",    how="inner", suffixes=("", "_pay"))
      .groupby(["city", "title"], as_index=False)
      .agg(total_revenue=("amount", "sum"))
      .sort_values("total_revenue", ascending=False)
      .head(5)
      .reset_index(drop=True)
)

print(max_kazancli_filmlerin_sehri)

-----------------------------------
37. EN AZ KAZANÇ SAĞLAYAN 5 FİLM HANGİ ŞEHİRDE BULUN
-----------------------------------

In [176]:
df = {}

for table_name in ["city", "address", "customer", "rental", "inventory", "film", "payment"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

film_revenue_by_city = (
    df["city"]
      .merge(df["address"],   on="city_id",      how="left",  suffixes=("", "_addr"))
      .merge(df["customer"],  on="address_id",   how="left",  suffixes=("", "_cust"))
      .merge(df["rental"],    on="customer_id",  how="left",  suffixes=("", "_rent"))
      .merge(df["inventory"], on="inventory_id", how="left",  suffixes=("", "_inv"))
      .merge(df["film"],      on="film_id",      how="left",  suffixes=("", "_film"))
      .merge(df["payment"],   on="rental_id",    how="left",  suffixes=("", "_pay"))
)

film_revenue_by_city["amount"] = film_revenue_by_city["amount"].fillna(0)

least_revenue_films_by_city = (
    film_revenue_by_city
      .groupby(["city", "title"], as_index=False)
      .agg(total_revenue=("amount", "sum"))
      .sort_values("total_revenue", ascending=True)
      .head(5)
      .reset_index(drop=True)
)

print(least_revenue_films_by_city.to_string(index=False))

               city             title  total_revenue
          Balurghat CURTAIN VIDEOTAPE            0.0
Aparecida de Goinia      WOMEN DORADO            0.0
              Merlo      BLADE POLISH            0.0
         Nagareyama    LAWLESS VISION            0.0
            Salinas    PRINCESS GIANT            0.0


-----------------------------------
38. EN FAZLA KİRALAMA YAPAN MÜŞTERİ HANGİ FİLMLERİ KİRALAMIŞ?
-----------------------------------

In [177]:
df = {}

for table_name in ["customer", "rental", "inventory", "film"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

top_renters = (
    df["customer"]
      .merge(df["rental"], on="customer_id", how="inner", suffixes=("", "_rent"))
      .groupby(["customer_id", "first_name", "last_name"], as_index=False)
      .agg(total_rentals=("rental_id", "count"))
      .sort_values("total_rentals", ascending=False)
      .head(3)
)

top_renters_movies = (
    df["rental"]
      .merge(top_renters[["customer_id"]], on="customer_id", how="inner")
      .merge(df["inventory"], on="inventory_id", how="inner", suffixes=("", "_inv"))
      .merge(df["film"], on="film_id", how="inner", suffixes=("", "_film"))
      .merge(df["customer"][["customer_id", "first_name", "last_name"]],
             on="customer_id", how="inner")
      .groupby(["first_name", "last_name", "title"], as_index=False)
      .agg(rental_count=("rental_id", "count"))
)

top_renters_movies = (
    top_renters_movies
      .sort_values(["first_name", "last_name", "rental_count"], ascending=[True, True, False])
      .groupby(["first_name", "last_name"], as_index=False)
      .head(3)
      .reset_index(drop=True)
)

print("Top renters (top 3) by total rentals:")
print(top_renters.to_string(index=False))

print("\nFilms rented by those top customers (top 3 per customer):")
print(top_renters_movies.to_string(index=False))

Top renters (top 3) by total rentals:
 customer_id first_name last_name  total_rentals
         148    ELEANOR      HUNT             46
         526       KARL      SEAL             45
         144      CLARA      SHAW             42

Films rented by those top customers (top 3 per customer):
first_name last_name                title  rental_count
     CLARA      SHAW      AIRPORT POLLOCK             1
     CLARA      SHAW ANACONDA CONFESSIONS             1
     CLARA      SHAW         ARIZONA BANG             1
   ELEANOR      HUNT     AFFAIR PREJUDICE             1
   ELEANOR      HUNT      AMERICAN CIRCUS             1
   ELEANOR      HUNT         ARIZONA BANG             1
      KARL      SEAL       WEDDING APOLLO             2
      KARL      SEAL        BETRAYED REAR             1
      KARL      SEAL     BIKINI BORROWERS             1


-----------------------------------
40. EN ÇOK KAZANÇ SAĞLAYAN MÜŞTERİ HANGİ FİLMLERİ KİRALAMIŞ?
-----------------------------------

In [179]:
df = {}
for table_name in ["customer", "rental", "inventory", "film", "payment"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

top_spender_id = (
    df["customer"]
      .merge(df["rental"],  on="customer_id", how="inner", suffixes=("", "_rent"))
      .merge(df["payment"], on="rental_id",   how="inner", suffixes=("", "_pay"))
      .groupby("customer_id", as_index=False)
      .agg(total_income=("amount", "sum"))
      .sort_values("total_income", ascending=False)
      .head(1)["customer_id"]
      .values[0]
)

cust_row = df["customer"].loc[df["customer"]["customer_id"] == top_spender_id, ["first_name", "last_name"]]
customer_name = f'{cust_row.iloc[0]["first_name"]} {cust_row.iloc[0]["last_name"]}' if not cust_row.empty else f"ID {top_spender_id}"

top_spender_movies = (
    df["customer"].query("customer_id == @top_spender_id")
      .merge(df["rental"],    on="customer_id",  how="inner", suffixes=("", "_rent"))
      .merge(df["inventory"], on="inventory_id", how="inner", suffixes=("", "_inv"))
      .merge(df["film"],      on="film_id",      how="inner", suffixes=("", "_film"))
      .merge(df["payment"],   on="rental_id",    how="inner", suffixes=("", "_pay"))
      .groupby("title", as_index=False)
      .agg(total_spent=("amount", "sum"))
      .sort_values("total_spent", ascending=False)
      .head(5)
      .reset_index(drop=True)
)

print(f"Top spender's rented films (customer: {customer_name}, id={top_spender_id}):")
print(top_spender_movies.to_string(index=False))

Top spender's rented films (customer: KARL SEAL, id=526):
           title  total_spent
FOOL MOCKINGBIRD        10.99
  STING PERSONAL         9.99
 TERMINATOR CLUB         8.99
  PICKUP DRIVING         8.99
     HIGH ENCINO         7.99


-----------------------------------
41. EN AZ KAZANÇ SAĞLAYAN MÜŞTERİ HANGİ FİLMLERİ KİRALAMIŞ?
-----------------------------------

In [180]:
df = {}
for table_name in ["customer", "rental", "inventory", "film", "payment"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

lowest_spender_id = (
    df["customer"]
      .merge(df["rental"],  on="customer_id", how="inner", suffixes=("", "_rent"))
      .merge(df["payment"], on="rental_id",   how="inner", suffixes=("", "_pay"))
      .groupby("customer_id", as_index=False)
      .agg(total_income=("amount", "sum"))
      .sort_values("total_income", ascending=True)
      .head(1)["customer_id"]
      .values[0]
)

lowest_spender_movies = (
    df["customer"].query("customer_id == @lowest_spender_id")
      .merge(df["rental"],    on="customer_id",  how="inner", suffixes=("", "_rent"))
      .merge(df["inventory"], on="inventory_id", how="inner", suffixes=("", "_inv"))
      .merge(df["film"],      on="film_id",      how="inner", suffixes=("", "_film"))
      .merge(df["payment"],   on="rental_id",    how="inner", suffixes=("", "_pay"))
      .groupby("title", as_index=False)
      .agg(total_spent=("amount", "sum"))
      .sort_values("total_spent", ascending=True)
      .head(7)
      .reset_index(drop=True)
)

print("Movies rented by the lowest-spending customer (top 7 by lowest total spent):")
print(lowest_spender_movies.to_string(index=False))

Movies rented by the lowest-spending customer (top 7 by lowest total spent):
                 title  total_spent
       ARMAGEDDON LOST         0.99
      CAMELOT VACATION         0.99
      EMPIRE MALKOVICH         0.99
       ILLUSION AMELIE         0.99
RESURRECTION SILVERADO         0.99
           TEXAS WATCH         0.99
            JADE BUNCH         2.99


-----------------------------------
42. EN AZ KAZANÇ SĞALAYAN MÜŞTERİ HANGİ TÜRDE EN FAZLA FİLM KİRALAMIŞ?
-----------------------------------

In [182]:
df = {}

for table_name in ["customer", "rental", "inventory", "film", "film_category", "category", "payment"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

lowest_spender_id = (
    df["customer"]
      .merge(df["rental"],  on="customer_id", how="inner", suffixes=("", "_rent"))
      .merge(df["payment"], on="rental_id",   how="inner", suffixes=("", "_pay"))
      .groupby("customer_id", as_index=False)
      .agg(total_income=("amount", "sum"))
      .sort_values("total_income", ascending=True)
      .head(1)["customer_id"]
      .values[0]
)

lowest_spender_top_genre = (
    df["customer"].query("customer_id == @lowest_spender_id")
      .merge(df["rental"],        on="customer_id",  how="inner", suffixes=("", "_rent"))
      .merge(df["inventory"],     on="inventory_id", how="inner", suffixes=("", "_inv"))
      .merge(df["film"],          on="film_id",      how="inner", suffixes=("", "_film"))
      .merge(df["film_category"], on="film_id",      how="inner", suffixes=("", "_fc"))
      .merge(df["category"],      on="category_id",  how="inner", suffixes=("", "_cat"))
      .groupby("name", as_index=False)
      .agg(rental_count=("rental_id", "count"))
      .rename(columns={"name": "genre"})
      .sort_values("rental_count", ascending=False)
      .head(1)
      .reset_index(drop=True)
)

print("Lowest-spending customer's most-rented genre:")
print(lowest_spender_top_genre.to_string(index=False))

Lowest-spending customer's most-rented genre:
 genre  rental_count
Sci-Fi             4


-----------------------------------
43. EN ÇOK KİRALANAN FİLM HANGİ ÇALIŞAN TARAFINDAN KİRALANMIŞ?
-----------------------------------

In [184]:
df = {}

for table_name in ["customer", "rental", "inventory", "film", "film_category", "category", "payment", "staff"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

rentals_per_film = (
    df["film"]
      .merge(df["inventory"], on="film_id", how="inner", suffixes=("", "_inv"))
      .merge(df["rental"],   on="inventory_id", how="inner", suffixes=("", "_rent"))
      .groupby(["film_id", "title"], as_index=False)
      .agg(rental_count=("rental_id", "count"))
)

max_count = rentals_per_film["rental_count"].max()
top_films = rentals_per_film[rentals_per_film["rental_count"] == max_count][["film_id", "title"]]

renting_staff = (
    top_films
      .merge(df["inventory"], on="film_id", how="inner", suffixes=("", "_inv"))
      .merge(df["rental"],   on="inventory_id", how="inner", suffixes=("", "_rent"))
      .merge(df["staff"],    on="staff_id",     how="inner", suffixes=("", "_staff"))
      .groupby(["first_name", "last_name", "title"], as_index=False)
      .agg(rental_count=("rental_id", "count"))
      .sort_values(["title", "rental_count"], ascending=[True, False])
      .reset_index(drop=True)
)

print(renting_staff.to_string(index=False))

first_name last_name              title  rental_count
       Jon  Stephens BUCKET BROTHERHOOD            18
      Mike   Hillyer BUCKET BROTHERHOOD            16


-----------------------------------
44. EN AZ KİRALANAN FİLM HANGİ ÇALIŞAN TARAFINDAN KİRALANMIŞ
-----------------------------------

In [189]:
df = {}

for table_name in ["staff", "rental", "inventory", "film", "payment"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

least_revenue_film_id = (
    df["film"].merge(df["inventory"], on="film_id", how="inner", suffixes=("", "_inv"))
              .merge(df["rental"],   on="inventory_id", how="inner", suffixes=("", "_rent"))
              .merge(df["payment"],  on="rental_id",    how="inner", suffixes=("", "_pay"))
              .groupby("film_id", as_index=False)["amount"].sum()
              .sort_values("amount")
              .iloc[0, 0]
)

renting_staff = (
    df["staff"].merge(df["rental"],    on="staff_id",     how="inner", suffixes=("", "_rent"))
               .merge(df["inventory"], on="inventory_id", how="inner", suffixes=("", "_inv"))
               .merge(df["payment"],   on="rental_id",    how="inner", suffixes=("", "_pay"))
               .query("film_id == @least_revenue_film_id")
               .groupby(["first_name", "last_name"], as_index=False)
               .agg(total_revenue=("amount", "sum"), rental_count=("rental_id", "count"))
               .sort_values(["total_revenue"], ascending=False)
)

title_map = df["film"][["film_id", "title"]].drop_duplicates()
renting_staff = renting_staff.assign(film_id=least_revenue_film_id) \
                             .merge(title_map, on="film_id", how="left") \
                             .drop(columns="film_id")

print(renting_staff.to_string(index=False))

first_name last_name  total_revenue  rental_count       title
       Jon  Stephens           2.97             3 TEXAS WATCH
      Mike   Hillyer           2.97             3 TEXAS WATCH


-----------------------------------
45. EN ÇOK KAZANÇ SAĞLAYAN FİLM HANGİ ÇALIŞAN TARAFINDAN KİRALANMIŞ?
-----------------------------------

In [191]:
df = {}
for table_name in ["staff", "rental", "inventory", "film", "payment", "store"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

film_revenue = (
    df["film"]
      .merge(df["inventory"], on="film_id", how="inner", suffixes=("", "_inv"))
      .merge(df["rental"],   on="inventory_id", how="inner", suffixes=("", "_rent"))
      .merge(df["payment"],  on="rental_id",    how="inner", suffixes=("", "_pay"))
      .groupby("film_id", as_index=False)
      .agg(total_revenue=("amount", "sum"))
)
top_grossing_film_id = film_revenue.loc[film_revenue["total_revenue"].idxmax(), "film_id"]

renting_staff = (
    df["staff"]
      .merge(df["rental"],    on="staff_id",     how="inner", suffixes=("", "_rent"))
      .merge(df["inventory"], on="inventory_id", how="inner", suffixes=("", "_inv"))
      .merge(df["film"],      on="film_id",      how="inner", suffixes=("", "_film"))
      .merge(df["payment"],   on="rental_id",    how="inner", suffixes=("", "_pay"))
      .query("film_id == @top_grossing_film_id")
      .groupby(["first_name", "last_name", "title"], as_index=False)
      .agg(total_revenue=("amount", "sum"), rental_count=("rental_id", "count"))
      .sort_values(["total_revenue", "rental_count"], ascending=[False, False])
      .reset_index(drop=True)
)

print("Staff who processed the top-grossing film:")
print(renting_staff.to_string(index=False))

first_name last_name            title  total_revenue  rental_count
       Jon  Stephens TELEGRAPH VOYAGE         147.83            17
      Mike   Hillyer TELEGRAPH VOYAGE          83.90            10


-----------------------------------
46. EN AZ KAZANÇ SAĞLAYAN FİLM HANGİ ÇALIŞAN TARAFINDAN KİRALANMIŞ?
-----------------------------------

In [192]:
df = {}

for table_name in ['staff', 'rental', 'inventory', 'film', 'payment', 'store']:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)
                                                                           for column in conn.execute(query).description])

least_grossing_film_id = (
    df["film"]
      .merge(df["inventory"], on="film_id",      how="inner", suffixes=("", "_inv"))
      .merge(df["rental"],   on="inventory_id",  how="inner", suffixes=("", "_rent"))
      .merge(df["payment"],  on="rental_id",     how="inner", suffixes=("", "_pay"))
      .groupby("film_id", as_index=False)["amount"].sum()
      .sort_values("amount", ascending=True)
      .iloc[0, 0]
)

staff_for_least = (
    df["staff"]
      .merge(df["rental"],    on="staff_id",     how="inner", suffixes=("", "_rent"))
      .merge(df["inventory"], on="inventory_id", how="inner", suffixes=("", "_inv"))
      .merge(df["payment"],   on="rental_id",    how="inner", suffixes=("", "_pay"))
      .query("film_id == @least_grossing_film_id")
      .merge(df["film"][["film_id", "title"]], on="film_id", how="left")
      .groupby(["first_name", "last_name", "title"], as_index=False)
      .agg(total_revenue=("amount", "sum"), rental_count=("rental_id", "count"))
      .sort_values(["total_revenue", "rental_count"], ascending=[False, False])
)

print("Staff who processed the least-grossing film:")
print(staff_for_least.to_string(index=False))

Staff who processed the least-grossing film:
first_name last_name       title  total_revenue  rental_count
       Jon  Stephens TEXAS WATCH           2.97             3
      Mike   Hillyer TEXAS WATCH           2.97             3


-----------------------------------
47. EN ÇOK KİRALANAN FİLM HANGİ MAĞAZADA KİRALANMIŞ?
-----------------------------------

In [196]:
df = {}

for table_name in ["staff", "rental", "inventory", "film", "payment", "store"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

most_rented_film_id = (
    df["film"]
      .merge(df["inventory"], on="film_id", suffixes=("", "_inv"))
      .merge(df["rental"],    on="inventory_id", suffixes=("", "_rent"))
      .groupby("film_id")["rental_id"]
      .count()
      .idxmax()
)

film_store = (
    df["store"]
      .merge(df["staff"],     on="store_id", suffixes=("", "_staff"))
      .merge(df["rental"],    on="staff_id", suffixes=("", "_rent"))
      .merge(df["inventory"], on="inventory_id", suffixes=("", "_inv"))
      .merge(df["film"],      on="film_id", suffixes=("", "_film"))
      .loc[lambda x: x["film_id"] == most_rented_film_id]
      .groupby(["store_id", "title"])
      .agg(rental_count=("rental_id", "count"))
      .reset_index()
      .rename(columns={"store_id": "store_id", "title": "film_title", "rental_count": "rental_count"})
)

print(film_store)

   store_id          film_title  rental_count
0         1  BUCKET BROTHERHOOD            16
1         2  BUCKET BROTHERHOOD            18


-----------------------------------
48. EN AZ KİRALANAN FİLM HANGİ MAĞAZADA KİRALANMIŞ?
-----------------------------------

In [197]:
df = {}

for table_name in ["staff", "rental", "inventory", "film", "payment", "store"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

least_rented_film_id = (
    df["film"]
      .merge(df["inventory"], on="film_id", suffixes=("", "_inv"))
      .merge(df["rental"],    on="inventory_id", suffixes=("", "_rent"))
      .groupby("film_id")["rental_id"]
      .count()
      .idxmin()
)

film_store = (
    df["store"]
      .merge(df["staff"],     on="store_id", suffixes=("", "_staff"))
      .merge(df["rental"],    on="staff_id", suffixes=("", "_rent"))
      .merge(df["inventory"], on="inventory_id", suffixes=("", "_inv"))
      .merge(df["film"],      on="film_id", suffixes=("", "_film"))
      .loc[lambda x: x["film_id"] == least_rented_film_id]
      .groupby(["store_id", "title"])
      .agg(rental_count=("rental_id", "count"))
      .reset_index()
      .rename(columns={"store_id": "store_id", "title": "film_title", "rental_count": "rental_count"})
)

print(film_store)

   store_id      film_title  rental_count
0         1  HARDLY ROBBERS             1
1         2  HARDLY ROBBERS             3


-----------------------------------
49. EN ÇOK KAZANÇ SAĞLAYAN FİLM HANGİ MAĞAZADA KİRALANMIŞ?
-----------------------------------

In [198]:
df = {}

for table_name in ["staff", "rental", "inventory", "film", "payment", "store"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

most_profitable_film_id = (
    df["film"]
      .merge(df["inventory"], on="film_id", suffixes=("", "_inv"))
      .merge(df["rental"],    on="inventory_id", suffixes=("", "_rent"))
      .merge(df["payment"],   on="rental_id",    suffixes=("", "_pay"))
      .groupby("film_id")["amount"].sum()
      .idxmax()
)

film_store = (
    df["store"]
      .merge(df["staff"],     on="store_id", suffixes=("", "_staff"))
      .merge(df["rental"],    on="staff_id", suffixes=("", "_rent"))
      .merge(df["inventory"], on="inventory_id", suffixes=("", "_inv"))
      .merge(df["film"],      on="film_id", suffixes=("", "_film"))
      .merge(df["payment"],   on="rental_id", suffixes=("", "_pay"))
      .loc[lambda x: x["film_id"] == most_profitable_film_id]
      .groupby(["store_id", "title"], as_index=False)
      .agg(total_revenue=("amount", "sum"))
      .rename(columns={"store_id": "store_id", "title": "film_title"})
)

print(film_store)

   store_id        film_title  total_revenue
0         1  TELEGRAPH VOYAGE          83.90
1         2  TELEGRAPH VOYAGE         147.83


-----------------------------------
50. EN AZ KAZANÇ SAĞLAYAN FİLM HANGİ MAĞAZADA KİRALANMIŞ?
-----------------------------------

In [200]:
df = {}

for table_name in ["staff", "rental", "inventory", "film", "payment", "store"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

least_profitable_film_id = (
    df["film"]
      .merge(df["inventory"], on="film_id", suffixes=("", "_inv"))
      .merge(df["rental"],    on="inventory_id", suffixes=("", "_rent"))
      .merge(df["payment"],   on="rental_id",    suffixes=("", "_pay"))
      .groupby("film_id")["amount"].sum()
      .idxmin()
)

film_store = (
    df["store"]
      .merge(df["staff"],     on="store_id", suffixes=("", "_staff"))
      .merge(df["rental"],    on="staff_id", suffixes=("", "_rent"))
      .merge(df["inventory"], on="inventory_id", suffixes=("", "_inv"))
      .merge(df["film"],      on="film_id", suffixes=("", "_film"))
      .merge(df["payment"],   on="rental_id", suffixes=("", "_pay"))
      .loc[lambda x: x["film_id"] == least_profitable_film_id]
      .groupby(["store_id", "title"], as_index=False)
      .agg(total_revenue=("amount", "sum"))
      .rename(columns={"title": "film_title"})
)

print(film_store)

   store_id        film_title  total_revenue
0         1  OKLAHOMA JUMANJI           3.96
1         2  OKLAHOMA JUMANJI           1.98


-----------------------------------
51. MÜŞTERİLERİN KİRALADIKLARI FİLMLERİN TOPLAM KİRALAMA SÜRESİ NE KADAR?
-----------------------------------

In [204]:
df = {}

for table_name in ["staff", "customer", "rental"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=[c[0] for c in cur.description])

df['rental']['rental_date'] = pd.to_datetime(df['rental']['rental_date'])
df['rental']['return_date'] = pd.to_datetime(df['rental']['return_date'])


total_rental_duration = (
    df["customer"]
      .merge(r[["customer_id", "rental_duration_days"]], on="customer_id", how="inner")
      .groupby(["first_name", "last_name"], as_index=False)
      .agg(total_rental_duration=("rental_duration_days", "sum"))
      .sort_values("total_rental_duration", ascending=False)
      .reset_index(drop=True)
)

print(total_rental_duration.to_string(index=False))

 first_name    last_name  total_rental_duration
       KARL         SEAL                  241.0
    ELEANOR         HUNT                  218.0
      CLARA         SHAW                  211.0
     RHONDA      KENNEDY                  209.0
     WESLEY         BULL                  203.0
      DAISY        BATES                  200.0
        TIM         CARY                  200.0
       JUNE      CARROLL                  195.0
     MARION       SNYDER                  195.0
     MARCIA         DEAN                  187.0
        ANA      BRADLEY                  184.0
       NEIL       RENNER                  182.0
     ARNOLD       HAVENS                  181.0
     CURTIS         IRBY                  181.0
   GERTRUDE     CASTILLO                  180.0
    BRANDON         HUEY                  179.0
      LOUIS        LEONE                  177.0
        SUE       PETERS                  176.0
       LENA       JENSEN                  174.0
       TROY      QUIGLEY                

-----------------------------------
52. EN ÇOK KİRALANAN TÜRDEKİ FİLMLER HANGİLERİ?
-----------------------------------

In [205]:
df = {}

for table_name in ["staff", "rental", "inventory", "film", "payment", "store", "customer", "category", "film_category"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=[c[0] for c in cur.description])

most_rented_categories = (
    df["film"]
      .merge(df["film_category"], on="film_id", suffixes=("", "_fc"))
      .merge(df["category"],      on="category_id", suffixes=("", "_cat"))
      .merge(df["inventory"],     on="film_id", suffixes=("", "_inv"))
      .merge(df["rental"],        on="inventory_id", suffixes=("", "_rent"))
      .groupby(["name", "title"])
      .agg(rental_count=("rental_id", "count"))
      .reset_index()
      .sort_values("rental_count", ascending=False)
      .head(3)
      .rename(columns={"name": "category"})
)

print(most_rented_categories)

    category                title  rental_count
909   Travel   BUCKET BROTHERHOOD            34
536  Foreign     ROCKETEER MOTHER            33
757      New  RIDGEMONT SUBMARINE            32


-----------------------------------
53. EN AZ KİRALANAN TÜRDEKİ FİLMLER HANGİLERİ?
-----------------------------------

In [208]:
df = {}

for table_name in ["staff", "rental", "inventory", "film", "payment", "store", "customer", "category", "film_category"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

least_rented_categories = (
    df["film"]
      .merge(df["film_category"], on="film_id", suffixes=("", "_fc"))
      .merge(df["category"],      on="category_id", suffixes=("", "_cat"))
      .merge(df["inventory"],     on="film_id", suffixes=("", "_inv"))
      .merge(df["rental"],        on="inventory_id", suffixes=("", "_rent"))
      .groupby(["name", "title"])
      .agg(rental_count=("rental_id", "count"))
      .reset_index()
      .rename(columns={"name": "genre"})
      .sort_values("rental_count", ascending=True)
      .head(3)
)

print("Least rented film categories:")
print(least_rented_categories)

Least rented film categories:
           genre           title  rental_count
315  Documentary  HARDLY ROBBERS             4
525      Foreign     MIXED DOORS             4
656       Horror     TRAIN BUNCH             4


-----------------------------------
54. HER MÜŞTERİ İÇİN TOPLAM ÖDEME MİKTARINI BULUN
-----------------------------------

In [207]:
df = {}

for table_name in ["staff", "rental", "inventory", "film", "payment", "store", "customer", "category", "film_category"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

total_payment = (
    df["customer"]
      .merge(df["rental"],  on="customer_id")
      .merge(df["payment"], on="rental_id")
      .groupby(["first_name", "last_name"], as_index=False)
      .agg(total_payment=("amount", "sum"))
)

print(total_payment)

    first_name last_name  total_payment
0        AARON     SELBY         110.76
1         ADAM     GOOCH         101.78
2       ADRIAN     CLARY          74.81
3        AGNES    BISHOP          98.77
4         ALAN      KAHN         124.74
..         ...       ...            ...
594     WILLIE   MARKHAM         101.75
595      WILMA  RICHARDS          91.80
596    YOLANDA    WEAVER         110.73
597     YVONNE   WATKINS          92.79
598    ZACHARY      HITE         146.69

[599 rows x 3 columns]


-----------------------------------
55. HANGİ FİLMLER EN UZUN SÜRE KİRALANMIŞ?
-----------------------------------

In [209]:
df = {}

for table_name in ["staff", "rental", "inventory", "film", "payment", "store", "customer", "category", "film_category"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

df["rental"]["rental_date"] = pd.to_datetime(df["rental"]["rental_date"])
df["rental"]["return_date"] = pd.to_datetime(df["rental"]["return_date"])

df["rental"]["rental_days"] = (df["rental"]["return_date"] - df["rental"]["rental_date"]).dt.days

longest_rented_films = (
    df["film"]
      .merge(df["inventory"], on="film_id")
      .merge(df["rental"],   on="inventory_id")
      .groupby("title", as_index=False)
      .agg(max_rental_days=("rental_days", "max"))
      .sort_values("max_rental_days", ascending=False)
)

print(longest_rented_films)

                 title  max_rental_days
0     ACADEMY DINOSAUR              9.0
653         PITY BOUND              9.0
551         MILE MULAN              9.0
552        MILLION ACE              9.0
553       MINDS TRUMAN              9.0
..                 ...              ...
800         SQUAD FISH              5.0
448    ITALIAN AFRICAN              5.0
341        GLORY TRACY              5.0
865     TRAFFIC HOBBIT              4.0
244  DUFFEL APOCALYPSE              4.0

[958 rows x 2 columns]


-----------------------------------
56. HANGİ FİLMLER EN KISA SÜRE KİRALANMIŞ?
-----------------------------------

In [210]:
df = {}

for table_name in ["staff", "rental", "inventory", "film", "payment", "store", "customer", "category", "film_category"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

df["rental"]["rental_date"] = pd.to_datetime(df["rental"]["rental_date"])
df["rental"]["return_date"] = pd.to_datetime(df["rental"]["return_date"])

df["rental"]["rental_days"] = (df["rental"]["return_date"] - df["rental"]["rental_date"]).dt.days

shortest_rented_films = (
    df["film"]
      .merge(df["inventory"], on="film_id")
      .merge(df["rental"],   on="inventory_id")
      .groupby("title", as_index=False)
      .agg(min_rental_days=("rental_days", "min"))
      .sort_values("min_rental_days", ascending=True)
)

print(shortest_rented_films)

                    title  min_rental_days
0        ACADEMY DINOSAUR              0.0
544       MERMAID INSECTS              0.0
546       METROPOLIS COMA              0.0
547  MICROCOSMOS PARADISE              0.0
548     MIDNIGHT WESTWARD              0.0
..                    ...              ...
337       GILBERT PELICAN              3.0
896       VANISHED GARDEN              3.0
223            DOLLS RAGE              3.0
764  SILVERADO GOLDFINGER              3.0
716           RUN PACIFIC              3.0

[958 rows x 2 columns]


-----------------------------------
57. MÜŞTERİLERİN KİRALADIĞI FİLMLER İÇİN ORTALAMA ÖDEME MİKTARINI BULUN
-----------------------------------

In [212]:
df = {}

for table_name in ["staff", "rental", "inventory", "film", "payment", "store", "customer", "category", "film_category"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

avg_payment = (
    df["customer"]
      .merge(df["rental"],  on="customer_id")
      .merge(df["payment"], on="rental_id")
      .groupby(["first_name", "last_name"], as_index=False)
      .agg(avg_payment=("amount", "mean"))
)

avg_payment["avg_payment"] = avg_payment["avg_payment"].round(5)

print(avg_payment)

    first_name last_name  avg_payment
0        AARON     SELBY      4.61500
1         ADAM     GOOCH      4.62636
2       ADRIAN     CLARY      3.93737
3        AGNES    BISHOP      4.29435
4         ALAN      KAHN      4.79769
..         ...       ...          ...
594     WILLIE   MARKHAM      4.07000
595      WILMA  RICHARDS      4.59000
596    YOLANDA    WEAVER      4.10111
597     YVONNE   WATKINS      4.41857
598    ZACHARY      HITE      4.73194

[599 rows x 3 columns]


-----------------------------------
58. EN ÇOK KİRALANAN FİLMLERİN ORTALAMA KİRALAMA SÜRESİ NEDİR?
-----------------------------------

In [214]:
df = {}

for table_name in ["staff", "rental", "inventory", "film", "payment", "store", "customer", "category", "film_category"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

df["rental"]["rental_date"] = pd.to_datetime(df["rental"]["rental_date"])
df["rental"]["return_date"] = pd.to_datetime(df["rental"]["return_date"])

df["rental"]["rental_days"] = (df["rental"]["return_date"] - df["rental"]["rental_date"]).dt.days

top_rented_films = (
    df["film"]
      .merge(df["inventory"], on="film_id", suffixes=("", "_inv"))
      .merge(df["rental"], on="inventory_id", suffixes=("", "_rent"))
      .assign(rental_days=lambda x: (x["return_date"] - x["rental_date"]).dt.days)
      .groupby("title", as_index=False)
      .agg(avg_rental_days=("rental_days", "mean"),
           rental_count=("rental_id", "count"))
      .sort_values(by="rental_count", ascending=False)
      .head(17)
)

print(top_rented_films[["title", "avg_rental_days", "rental_count"]])

                   title  avg_rental_days  rental_count
96    BUCKET BROTHERHOOD         4.411765            34
705     ROCKETEER MOTHER         4.939394            33
697  RIDGEMONT SUBMARINE         5.580645            32
361       GRIT CLOCKWORK         4.812500            32
465       JUGGLER HARDLY         5.064516            32
312       FORWARD TEMPLE         5.125000            32
733        SCALAWAG DUCK         3.593750            32
957            ZORRO ARK         4.096774            31
853       TIMBERLAND SKY         5.290323            31
29         APACHE DIVINE         3.709677            31
395         HOBBIT ALIEN         5.161290            31
348    GOODFELLAS SALUTE         4.838710            31
594         NETWORK PEAK         5.000000            31
930            WIFE TURN         4.161290            31
719      RUSH GOODFELLAS         4.032258            31
702         ROBBERS JOON         4.354839            31
755          SHOCK CABIN         4.964286       

-----------------------------------
59. EN AZ KİRALANAN FİLMLERİN ORTALAMA KİRALAMA SÜRESİ NEDİR?
-----------------------------------

In [215]:
df = {}

for table_name in ["staff", "rental", "inventory", "film", "payment", "store", "customer", "category", "film_category"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

df["rental"]["rental_date"] = pd.to_datetime(df["rental"]["rental_date"])
df["rental"]["return_date"] = pd.to_datetime(df["rental"]["return_date"])

df["rental"]["rental_days"] = (df["rental"]["return_date"] - df["rental"]["rental_date"]).dt.days

least_rented_films = (
    df["film"]
      .merge(df["inventory"], on="film_id", suffixes=("", "_inv"))
      .merge(df["rental"], on="inventory_id", suffixes=("", "_rent"))
      .assign(rental_days=lambda x: (x["return_date"] - x["rental_date"]).dt.days)
      .groupby("title", as_index=False)
      .agg(avg_rental_days=("rental_days", "mean"),
           rental_count=("rental_id", "count"))
      .sort_values(by="rental_count", ascending=True)
      .head(17)
)

print(least_rented_films[["title", "avg_rental_days", "rental_count"]])

                  title  avg_rental_days  rental_count
558         MIXED DOORS             5.75             4
866         TRAIN BUNCH             3.00             4
378      HARDLY ROBBERS             6.50             4
585  MUSSOLINI SPOILERS             3.60             5
315   FREEDOM CLEOPATRA             2.40             5
669        PRIVATE DROP             4.20             5
293        FEVER EMPIRE             5.20             5
168   CONSPIRACY SPIRIT             3.60             5
532     MANNEQUIN WORST             4.00             5
435     INFORMER DOUBLE             2.80             5
417        HUNTER ALTER             6.20             5
323     FULL FLATLINERS             3.40             5
747         SEVEN SWARM             4.00             5
865      TRAFFIC HOBBIT             2.80             5
341         GLORY TRACY             3.40             5
100         BUNCH MINDS             5.40             5
87     BRAVEHEART HUMAN             3.40             5


-----------------------------------
60. EN ÇOK KAZANÇ SAĞLAYAN MÜŞTERİLERİN ORTALAMA ÖDEME MİKTARI NEDİR?
-----------------------------------

In [216]:
df = {}

for table_name in ["staff", "rental", "inventory", "film", "payment", "store", "customer", "category", "film_category"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

avg_payment = (
    df["customer"]
      .merge(df["rental"], on="customer_id", suffixes=("", "_r"))
      .merge(df["payment"], on="rental_id", suffixes=("", "_p"))
      .groupby(["first_name", "last_name"], as_index=False)
      .agg(avg_payment=("amount", "mean"), total_payment=("amount", "sum"))
      .sort_values(by="total_payment", ascending=False)
      .head(12)
)

print(avg_payment[["first_name", "last_name", "avg_payment"]])

    first_name last_name  avg_payment
318       KARL      SEAL     4.923333
175    ELEANOR      HUNT     4.707391
105      CLARA      SHAW     4.656667
474     RHONDA   KENNEDY     4.990000
389     MARION    SNYDER     4.990000
556      TOMMY   COLLAZO     4.911053
590     WESLEY      BULL     4.440000
551        TIM      CARY     4.502821
379     MARCIA      DEAN     4.180476
21         ANA   BRADLEY     5.137059
315       JUNE   CARROLL     4.692703
151      DIANE   COLLINS     4.847143


-----------------------------------
61. EN AZ KAZANÇ SAĞLAYAN MÜŞTERİLERİN ORTALAMA ÖDEME MİKTARI NEDİR?
-----------------------------------

In [229]:
df = {}

for table_name in ["staff", "rental", "inventory", "film", "payment", "store", "customer", "category", "film_category"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

least_profitable_customers = (
    df["customer"]
      .merge(df["rental"], on="customer_id", suffixes=("", "_r"))
      .merge(df["payment"], on="rental_id", suffixes=("", "_p"))
      .groupby(["first_name", "last_name"], as_index=False)
      .agg(avg_payment=("amount", "mean"), total_payment=("amount", "sum"))
      .sort_values(by="total_payment", ascending=True)
      .head(12)
)

print(least_profitable_customers[["first_name", "last_name", "avg_payment"]])

    first_name last_name  avg_payment
83    CAROLINE    BOWMAN     3.390000
351      LEONA    OBRIEN     3.632857
71       BRIAN     WYMAN     4.406667
296     JOHNNY    TURPIN     3.042632
33       ANNIE   RUSSELL     3.267778
319  KATHERINE    RIVERA     4.204286
550    TIFFANY    JORDAN     4.275714
28       ANITA   MORALES     4.190000
401     MATTIE   HOFFMAN     2.944545
334       KIRK   STCLAIR     3.411053
356     LESTER     KRAUS     4.115000
6      ALBERTO   HENNING     3.180476


-----------------------------------
62. MAĞAZALARDAKİ TOPLAM KİRALAMA SÜRESİNİ BULUN
-----------------------------------

In [219]:
df = {}

for table_name in ["staff", "rental", "inventory", "film", "payment", "store", "customer", "category", "film_category"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

df["rental"]["rental_date"] = pd.to_datetime(df["rental"]["rental_date"])
df["rental"]["return_date"] = pd.to_datetime(df["rental"]["return_date"])

df["rental"]["rental_duration"] = (df["rental"]["return_date"] - df["rental"]["rental_date"]).dt.days

total_duration = (
    df["store"]
      .merge(df["staff"], on="store_id", how="inner")
      .merge(df["rental"], on="staff_id", how="inner")
      .groupby("store_id", as_index=False)
      .agg(total_rental_days=("rental_duration", "sum"))
)

print(total_duration)

   store_id  total_rental_days
0         1            35897.0
1         2            35889.0


-----------------------------------
63. EN UZUN SÜRE KİRALANAN FİLM HANGİ MAĞAZADA KİRALANMIŞ?
-----------------------------------

In [221]:
df = {}

for table_name in ["staff", "rental", "inventory", "film", "payment", "store", "customer", "category", "film_category"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

df["rental"]["rental_date"] = pd.to_datetime(df["rental"]["rental_date"])
df["rental"]["return_date"] = pd.to_datetime(df["rental"]["return_date"])

df["rental"]["rental_duration"] = (df["rental"]["return_date"] - df["rental"]["rental_date"]).dt.days

longest_rented_film = (
    df["store"]
      .merge(df["staff"], on="store_id", suffixes=("", "_staff"))
      .merge(df["rental"], on="staff_id", suffixes=("", "_rental"))
      .merge(df["inventory"], on="inventory_id", suffixes=("", "_inv"))
      .merge(df["film"], on="film_id", suffixes=("", "_film"))
      .groupby(["store_id", "title"])
      .agg(max_rental_days=("rental_duration", "max"))
      .reset_index()
      .sort_values(by="max_rental_days", ascending=False)
      .head(1)
)

print(longest_rented_film)

   store_id             title  max_rental_days
0         1  ACADEMY DINOSAUR              9.0


-----------------------------------
64. EN KISA SÜREDE KİRALANAN FİLM HANGİ MAĞAZADA KİRALANMIŞ?
-----------------------------------

In [222]:
df = {}

for table_name in ["staff", "rental", "inventory", "film", "payment", "store", "customer", "category", "film_category"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

df["rental"]["rental_date"] = pd.to_datetime(df["rental"]["rental_date"])
df["rental"]["return_date"] = pd.to_datetime(df["rental"]["return_date"])

df["rental"]["rental_duration"] = (df["rental"]["return_date"] - df["rental"]["rental_date"]).dt.days

shortest_rented_film = (
    df["store"]
      .merge(df["staff"], on="store_id", suffixes=("", "_staff"))
      .merge(df["rental"], on="staff_id", suffixes=("", "_rental"))
      .merge(df["inventory"], on="inventory_id", suffixes=("", "_inv"))
      .merge(df["film"], on="film_id", suffixes=("", "_film"))
      .groupby(["store_id", "title"])
      .agg(min_rental_duration=("rental_duration", "min"))
      .reset_index()
      .sort_values(by="min_rental_duration", ascending=True)
      .head(1)
)

print(shortest_rented_film[["store_id", "title", "min_rental_duration"]])

   store_id             title  min_rental_duration
0         1  ACADEMY DINOSAUR                  0.0


-----------------------------------
65. HER FİLMİN ORTALAMA KİRALAMA SÜRESİ NEDİR?
-----------------------------------

In [223]:
df = {}

for table_name in ["staff", "rental", "inventory", "film", "payment", "store", "customer", "category", "film_category"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

df["rental"]["rental_date"] = pd.to_datetime(df["rental"]["rental_date"])
df["rental"]["return_date"] = pd.to_datetime(df["rental"]["return_date"])

df["rental"]["rental_duration"] = (df["rental"]["return_date"] - df["rental"]["rental_date"]).dt.days

avg_rental_duration = (
    df["film"]
      .merge(df["inventory"], on="film_id")
      .merge(df["rental"], on="inventory_id")
      .assign(rental_days=lambda x: (pd.to_datetime(x["return_date"]) - pd.to_datetime(x["rental_date"])).dt.days)
      .groupby("title")
      .agg(avg_rental_days=("rental_days", "mean"))
      .reset_index()
)

print(avg_rental_duration)

                 title  avg_rental_days
0     ACADEMY DINOSAUR         4.545455
1       ACE GOLDFINGER         5.333333
2     ADAPTATION HOLES         2.833333
3     AFFAIR PREJUDICE         4.363636
4          AFRICAN EGG         6.727273
..                 ...              ...
953     YOUNG LANGUAGE         4.000000
954         YOUTH KICK         4.666667
955       ZHIVAGO CORE         5.250000
956  ZOOLANDER FICTION         5.176471
957          ZORRO ARK         4.096774

[958 rows x 2 columns]


-----------------------------------
66. HANGİ FİLMLER EN ÇOK KİRALANAN KATEGORİDE?
-----------------------------------

In [224]:
df = {}

for table_name in ["staff", "rental", "inventory", "film", "payment", "store", "customer", "category", "film_category"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

popular_film_categories = (
    df["category"]
      .merge(df["film_category"], on="category_id", how="inner", suffixes=("", "_fc"))
      .merge(df["film"], on="film_id", how="inner", suffixes=("", "_film"))
      .merge(df["inventory"], on="film_id", how="inner", suffixes=("", "_inv"))
      .merge(df["rental"], on="inventory_id", how="inner", suffixes=("", "_r"))
      .groupby(["name", "title"])
      .agg(rental_count=("rental_id", "count"))
      .reset_index()
      .rename(columns={"name": "genre"})
      .sort_values(by="rental_count", ascending=False)
)

print(popular_film_categories)

           genre                title  rental_count
909       Travel   BUCKET BROTHERHOOD            34
536      Foreign     ROCKETEER MOTHER            33
757          New  RIDGEMONT SUBMARINE            32
572        Games       GRIT CLOCKWORK            32
93     Animation       JUGGLER HARDLY            32
..           ...                  ...           ...
513      Foreign      INFORMER DOUBLE             5
570        Games          GLORY TRACY             5
656       Horror          TRAIN BUNCH             4
525      Foreign          MIXED DOORS             4
315  Documentary       HARDLY ROBBERS             4

[958 rows x 3 columns]


-----------------------------------
67. HANGİ FİLMLER EN AZ KİRALANAN KATEGORİDE?
-----------------------------------

In [228]:
df = {}

for table_name in ["staff", "rental", "inventory", "film", "payment", "store", "customer", "category", "film_category"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

least_film_categories = (
    df["category"]
      .merge(df["film_category"], on="category_id", how="inner", suffixes=("", "_fc"))
      .merge(df["film"], on="film_id", how="inner", suffixes=("", "_film"))
      .merge(df["inventory"], on="film_id", how="inner", suffixes=("", "_inv"))
      .merge(df["rental"], on="inventory_id", how="inner", suffixes=("", "_r"))
      .groupby(["name", "title"])
      .agg(rental_count=("rental_id", "count"))
      .reset_index()
      .rename(columns={"name": "genre"})
      .sort_values(by="rental_count", ascending=True)
)

print(least_film_categories)

           genre                title  rental_count
315  Documentary       HARDLY ROBBERS             4
525      Foreign          MIXED DOORS             4
656       Horror          TRAIN BUNCH             4
746          New      MANNEQUIN WORST             5
566        Games         FEVER EMPIRE             5
..           ...                  ...           ...
572        Games       GRIT CLOCKWORK            32
568        Games       FORWARD TEMPLE            32
757          New  RIDGEMONT SUBMARINE            32
536      Foreign     ROCKETEER MOTHER            33
909       Travel   BUCKET BROTHERHOOD            34

[958 rows x 3 columns]


-----------------------------------
68. HANGİ MAĞAZALARDA EN ÇOK FİLM KİRALANMIŞ?
-----------------------------------

In [230]:
df = {}

for table_name in ["staff", "rental", "inventory", "film", "payment", "store", "customer", "category", "film_category"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

most_rented_genres = (
    df["category"]
      .merge(df["film_category"], on="category_id", suffixes=("", "_fc"))
      .merge(df["film"], on="film_id", suffixes=("", "_film"))
      .merge(df["inventory"], on="film_id", suffixes=("", "_inv"))
      .merge(df["rental"], on="inventory_id", suffixes=("", "_r"))
      .groupby("name")
      .agg(total_rentals=("rental_id", "count"))
      .reset_index()
      .rename(columns={"name": "genre"})
      .sort_values(by="total_rentals", ascending=False)
)

print(most_rented_genres)

          genre  total_rentals
14       Sports           1179
1     Animation           1166
0        Action           1112
13       Sci-Fi           1101
7        Family           1096
6         Drama           1060
5   Documentary           1050
8       Foreign           1033
9         Games            969
2      Children            945
4        Comedy            941
12          New            940
3      Classics            939
10       Horror            846
15       Travel            837
11        Music            830


-----------------------------------
69. HANGİ MAĞAZALARDA EN AZ FİLM KİRALANMIŞ?
-----------------------------------

In [233]:
df = {}

for table_name in ["staff", "rental", "inventory", "film", "payment", "store", "customer", "category", "film_category"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

least_rented_films = (
    df["film"]
      .merge(df["inventory"], on="film_id")
      .merge(df["rental"], on="inventory_id")
      .groupby("title")
      .agg(total_rentals=("rental_id", "count"))
      .reset_index()
      .sort_values(by="total_rentals", ascending=True)
)

print(least_rented_films)

                  title  total_rentals
558         MIXED DOORS              4
866         TRAIN BUNCH              4
378      HARDLY ROBBERS              4
585  MUSSOLINI SPOILERS              5
315   FREEDOM CLEOPATRA              5
..                  ...            ...
361      GRIT CLOCKWORK             32
733       SCALAWAG DUCK             32
312      FORWARD TEMPLE             32
705    ROCKETEER MOTHER             33
96   BUCKET BROTHERHOOD             34

[958 rows x 2 columns]


-----------------------------------
70. HANGİ AKTÖRLER EN ÇOK FİLMDE ROL ALMIŞ?
-----------------------------------

In [234]:
df = {}
for table_name in [
    "staff", "rental", "inventory", "film", "payment", "store",
    "customer", "category", "film_category", "actor", "film_actor"
]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

top_actors = (
    df["actor"]
      .merge(df["film_actor"], on="actor_id", how="inner")
      .groupby(["first_name", "last_name"])
      .agg(film_count=("film_id", "count"))
      .reset_index()
      .sort_values(by="film_count", ascending=False)
)

print(top_actors)

    first_name  last_name  film_count
180      SUSAN      DAVIS          54
65        GINA  DEGENERES          42
190     WALTER       TORN          41
123       MARY     KEITEL          40
125    MATTHEW     CARREY          39
..         ...        ...         ...
177      SISSY   SOBIESKI          18
103      JULIA  ZELLWEGER          16
101      JULIA    FAWCETT          15
99        JUDY       DEAN          15
51       EMILY        DEE          14

[199 rows x 3 columns]


-----------------------------------
71. HANGİ AKTÖRLER EN AZ FİLMDE ROL ALMIŞ?
-----------------------------------

In [235]:
df = {}

for table_name in [
    "staff", "rental", "inventory", "film", "payment", "store",
    "customer", "category", "film_category", "actor", "film_actor"
]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

least_actors = (
    df["actor"]
      .merge(df["film_actor"], on="actor_id", how="inner")
      .groupby(["first_name", "last_name"])
      .agg(film_count=("film_id", "count"))
      .reset_index()
      .sort_values(by="film_count", ascending=True)
)

print(least_actors)

    first_name  last_name  film_count
51       EMILY        DEE          14
99        JUDY       DEAN          15
101      JULIA    FAWCETT          15
103      JULIA  ZELLWEGER          16
177      SISSY   SOBIESKI          18
..         ...        ...         ...
125    MATTHEW     CARREY          39
123       MARY     KEITEL          40
190     WALTER       TORN          41
65        GINA  DEGENERES          42
180      SUSAN      DAVIS          54

[199 rows x 3 columns]


-----------------------------------
72. HANGİ AKTÖRLERİN OYNADIĞI FİLMLER EN ÇOK KİRALANMIŞ?
-----------------------------------

In [238]:
df = {}
for table_name in [
    "staff", "rental", "inventory", "film", "payment", "store",
    "customer", "category", "film_category", "actor", "film_actor"
]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

for t in ["actor", "film_actor", "film", "inventory", "rental"]:
    if "last_update" in df[t].columns:
        df[t] = df[t].drop(columns=["last_update"])

actors_rentals = (
    df["actor"]
      .merge(df["film_actor"], on="actor_id", how="inner")
      .merge(df["film"], on="film_id", how="inner")
      .merge(df["inventory"], on="film_id", how="inner")
      .merge(df["rental"], on="inventory_id", how="inner")
      .groupby(["actor_id", "first_name", "last_name", "title"])
      .agg(total_rentals=("rental_id", "count"))
      .reset_index()
      .sort_values(by="total_rentals", ascending=False)
      .head(10)
)

print("Most rented films and their actors:")
print(actors_rentals[["first_name", "last_name", "title", "total_rentals"]])

Most rented films and their actors:
     first_name    last_name               title  total_rentals
1283       GARY      PHOENIX  BUCKET BROTHERHOOD             34
784         TIM      HACKMAN  BUCKET BROTHERHOOD             34
2274   CHARLIZE        DENCH  BUCKET BROTHERHOOD             34
2352    KIRSTEN       AKROYD  BUCKET BROTHERHOOD             34
5042       BURT       TEMPLE  BUCKET BROTHERHOOD             34
629         RIP     CRAWFORD  BUCKET BROTHERHOOD             34
862        JUDY         DEAN    ROCKETEER MOTHER             33
3125     WARREN      JACKMAN    ROCKETEER MOTHER             33
5102      JAYNE  SILVERSTONE    ROCKETEER MOTHER             33
1055        TOM      MIRANDA    ROCKETEER MOTHER             33


-----------------------------------
73. HANGİ AKTÖRLERİN OYNADIĞI FİLMLER EN AZ KİRALANMIŞ?
-----------------------------------

In [239]:
df = {}

for table_name in ["staff", "rental", "inventory", "film", "payment", "store", "customer", "category", "film_category", "actor", "film_actor"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=[c[0] for c in cur.description])

for t in ["actor", "film_actor", "film", "inventory", "rental"]:
    if "last_update" in df[t].columns:
        df[t] = df[t].drop(columns=["last_update"])

least_rented_actor_films = (
    df["actor"]
      .merge(df["film_actor"], on="actor_id", how="inner")
      .merge(df["film"],       on="film_id",  how="inner")
      .merge(df["inventory"],  on="film_id",  how="inner")
      .merge(df["rental"],     on="inventory_id", how="inner")
      .groupby(["actor_id", "first_name", "last_name", "title"], as_index=False)
      .agg(total_rentals=("rental_id", "count"))
      .sort_values("total_rentals", ascending=True)
      .head(10)
)

print(least_rented_actor_films[["first_name", "last_name", "title", "total_rentals"]].to_string(index=False))

first_name   last_name          title  total_rentals
     GRETA      MALDEN HARDLY ROBBERS              4
      ADAM      HOPPER    TRAIN BUNCH              4
  PENELOPE      CRONYN    TRAIN BUNCH              4
  MICHELLE MCCONAUGHEY    MIXED DOORS              4
   MICHAEL      BOLGER    MIXED DOORS              4
    ALBERT       NOLTE    MIXED DOORS              4
     JULIA     MCQUEEN    TRAIN BUNCH              4
       IAN       TANDY    MIXED DOORS              4
      KARL       BERRY HARDLY ROBBERS              4
       DAN      STREEP    MIXED DOORS              4


-----------------------------------
74. HANGİ KATEGORİLERDE EN FAZLA AKTÖR OYNAMIŞ?
-----------------------------------

In [240]:
df = {}

for table_name in ["staff", "rental", "inventory", "film", "payment", "store", "customer", "category", "film_category", "actor", "film_actor"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=[c[0] for c in cur.description])

actors_per_category = (
    df["category"]
      .merge(df["film_category"], on="category_id", how="inner", suffixes=("", "_fc"))
      .merge(df["film"],         on="film_id",     how="inner", suffixes=("", "_film"))
      .merge(df["film_actor"],   on="film_id",     how="inner", suffixes=("", "_fa"))
      .groupby("name", as_index=False)
      .agg(actor_count=("actor_id", "nunique"))
      .rename(columns={"name": "genre"})
      .sort_values("actor_count", ascending=False)
)

print(actors_per_category)

          genre  actor_count
14       Sports          182
8       Foreign          175
12          New          169
5   Documentary          168
13       Sci-Fi          167
0        Action          166
1     Animation          166
15       Travel          166
7        Family          164
2      Children          163
3      Classics          162
6         Drama          162
10       Horror          156
9         Games          150
4        Comedy          147
11        Music          144


-----------------------------------
75. HANGİ KATEGORİLERDE EN AZ AKTÖR OYNAMIŞ?
-----------------------------------

In [241]:
df = {}

for table_name in [
    "staff", "rental", "inventory", "film", "payment", "store",
    "customer", "category", "film_category", "actor", "film_actor"
]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=[c[0] for c in cur.description])

actors_per_category = (
    df["category"]
      .merge(df["film_category"], on="category_id", how="inner", suffixes=("", "_fc"))
      .merge(df["film"],         on="film_id",     how="inner", suffixes=("", "_film"))
      .merge(df["film_actor"],   on="film_id",     how="inner", suffixes=("", "_fa"))
      .groupby("name", as_index=False)
      .agg(actor_count=("actor_id", "nunique"))
      .rename(columns={"name": "genre"})
      .sort_values("actor_count", ascending=True)
)

print(actors_per_category)

          genre  actor_count
11        Music          144
4        Comedy          147
9         Games          150
10       Horror          156
3      Classics          162
6         Drama          162
2      Children          163
7        Family          164
0        Action          166
1     Animation          166
15       Travel          166
13       Sci-Fi          167
5   Documentary          168
12          New          169
8       Foreign          175
14       Sports          182


-----------------------------------
76. HANGİ KATEGORİLERDE OYNAYAN FİLMLER EN ÇOK KİRALANMIŞ?
-----------------------------------

In [243]:
df = {}

for table_name in ["category", "film_category", "film", "inventory", "rental"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=[c[0] for c in cur.description])

popular_categories = (
    df["category"]
      .merge(df["film_category"], on="category_id", suffixes=("", "_fc"))
      .merge(df["film"], on="film_id", suffixes=("", "_film"))
      .merge(df["inventory"], on="film_id", suffixes=("", "_inv"))
      .merge(df["rental"], on="inventory_id", suffixes=("", "_r"))
      .groupby("name", as_index=False)
      .agg(rental_count=("rental_id", "count"))
      .rename(columns={"name": "genre"})
      .sort_values(by="rental_count", ascending=False)
)

print(popular_categories)

          genre  rental_count
14       Sports          1179
1     Animation          1166
0        Action          1112
13       Sci-Fi          1101
7        Family          1096
6         Drama          1060
5   Documentary          1050
8       Foreign          1033
9         Games           969
2      Children           945
4        Comedy           941
12          New           940
3      Classics           939
10       Horror           846
15       Travel           837
11        Music           830


-----------------------------------
77. HANGİ KATEGORİLERDE OYNAYAN FİLMLER EN AZ KİRALANMIŞ?
-----------------------------------

In [244]:
df = {}

for table_name in ["category", "film_category", "film", "inventory", "rental"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=[c[0] for c in cur.description])

least_rented_categories = (
    df["category"]
      .merge(df["film_category"], on="category_id", suffixes=("", "_fc"))
      .merge(df["film"], on="film_id", suffixes=("", "_film"))
      .merge(df["inventory"], on="film_id", suffixes=("", "_inv"))
      .merge(df["rental"], on="inventory_id", suffixes=("", "_r"))
      .groupby("name", as_index=False)
      .agg(rental_count=("rental_id", "count"))
      .rename(columns={"name": "genre"})
      .sort_values(by="rental_count", ascending=True)
)

print(least_rented_categories)

          genre  rental_count
11        Music           830
15       Travel           837
10       Horror           846
3      Classics           939
12          New           940
4        Comedy           941
2      Children           945
9         Games           969
8       Foreign          1033
5   Documentary          1050
6         Drama          1060
7        Family          1096
13       Sci-Fi          1101
0        Action          1112
1     Animation          1166
14       Sports          1179


-----------------------------------
78. EN FAZLA KAZANÇ SAĞLAYAN KATEGORİLER HANGİLERİ?
-----------------------------------

In [245]:
df = {}

for table_name in ["category", "film_category", "film", "inventory", "rental"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=[c[0] for c in cur.description])

max_revenue_categories = (
    df["category"]
      .merge(df["film_category"], on="category_id", how="inner", suffixes=("", "_fc"))
      .merge(df["film"], on="film_id", how="inner", suffixes=("", "_film"))
      .merge(df["inventory"], on="film_id", how="inner", suffixes=("", "_inv"))
      .merge(df["rental"], on="inventory_id", how="inner", suffixes=("", "_r"))
      .groupby("name", as_index=False)
      .agg(total_revenue=("rental_rate", lambda x: x.sum() * len(x)))
      .rename(columns={"name": "genre"})
      .sort_values(by="total_revenue", ascending=False)
)

print(max_revenue_categories)

          genre  total_revenue
14       Sports     4264690.59
1     Animation     3752584.44
13       Sci-Fi     3622278.99
6         Drama     3575804.00
0        Action     3299170.56
7        Family     3243107.84
8       Foreign     3151342.11
9         Games     2939277.39
4        Comedy     2907304.19
5   Documentary     2884875.00
12          New     2730324.00
2      Children     2401764.75
3      Classics     2326475.79
15       Travel     2324039.31
10       Horror     2219514.84
11        Music     2109611.00


-----------------------------------
79. EN AZ KAZANÇ SAĞLAYAN KATEGORİLER HANGİLERİ?
-----------------------------------

In [246]:
df = {}

for table_name in ["category", "film_category", "film", "inventory", "rental"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=[c[0] for c in cur.description])

max_revenue_categories = (
    df["category"]
      .merge(df["film_category"], on="category_id", how="inner", suffixes=("", "_fc"))
      .merge(df["film"], on="film_id", how="inner", suffixes=("", "_film"))
      .merge(df["inventory"], on="film_id", how="inner", suffixes=("", "_inv"))
      .merge(df["rental"], on="inventory_id", how="inner", suffixes=("", "_r"))
      .groupby("name", as_index=False)
      .agg(total_revenue=("rental_rate", lambda x: x.sum() * len(x)))
      .rename(columns={"name": "genre"})
      .sort_values(by="total_revenue", ascending=True)
)

print(max_revenue_categories)

          genre  total_revenue
11        Music     2109611.00
10       Horror     2219514.84
15       Travel     2324039.31
3      Classics     2326475.79
2      Children     2401764.75
12          New     2730324.00
5   Documentary     2884875.00
4        Comedy     2907304.19
9         Games     2939277.39
8       Foreign     3151342.11
7        Family     3243107.84
0        Action     3299170.56
6         Drama     3575804.00
13       Sci-Fi     3622278.99
1     Animation     3752584.44
14       Sports     4264690.59


-----------------------------------
80. EN ÇOK FİLM KİRALAYAN MÜŞTERİLERİN ORTALAMA ÖDEME MİKTARI NEDİR?
-----------------------------------

In [247]:
df = {}
for table_name in ["customer", "rental", "payment"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

top_customers_avg_payment = (
    df["customer"]
      .merge(df["rental"], on="customer_id", how="inner")
      .merge(df["payment"], on="rental_id", how="inner")
      .groupby(["first_name", "last_name"], as_index=False)
      .agg(
          avg_payment=("amount", "mean"),
          rental_count=("rental_id", "count")
      )
      .sort_values(by="rental_count", ascending=False)
      .head(17)
)

print(top_customers_avg_payment)

    first_name last_name  avg_payment  rental_count
175    ELEANOR      HUNT     4.707391            46
318       KARL      SEAL     4.923333            45
379     MARCIA      DEAN     4.180476            42
105      CLARA      SHAW     4.656667            42
536      TAMMY   SANDERS     3.794878            41
590     WESLEY      BULL     4.440000            40
531        SUE    PETERS     3.865000            40
389     MARION    SNYDER     4.990000            39
551        TIM      CARY     4.502821            39
474     RHONDA   KENNEDY     4.990000            39
176  ELIZABETH     BROWN     3.805789            38
125      DAISY     BATES     4.279474            38
123     CURTIS      IRBY     4.411053            38
556      TOMMY   COLLAZO     4.911053            38
315       JUNE   CARROLL     4.692703            37
66     BRANDON      HUEY     4.125135            37
180      ELSIE    KELLEY     3.827838            37


-----------------------------------
81. EN AZ FİLM KİRALAYAN MÜŞTERİLERİN ORTALAMA ÖDEME MİKTARI NEDİR?
-----------------------------------

In [248]:
df = {}
for table_name in ["customer", "rental", "payment"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

top_customers_avg_payment = (
    df["customer"]
      .merge(df["rental"], on="customer_id", how="inner")
      .merge(df["payment"], on="rental_id", how="inner")
      .groupby(["first_name", "last_name"], as_index=False)
      .agg(
          avg_payment=("amount", "mean"),
          rental_count=("rental_id", "count")
      )
      .sort_values(by="rental_count", ascending=True)
      .head(17)
)

print(top_customers_avg_payment)

    first_name  last_name  avg_payment  rental_count
71       BRIAN      WYMAN     4.406667            12
550    TIFFANY     JORDAN     4.275714            14
351      LEONA     OBRIEN     3.632857            14
319  KATHERINE     RIVERA     4.204286            14
83    CAROLINE     BOWMAN     3.390000            15
28       ANITA    MORALES     4.190000            15
277     JEROME     KENYON     4.615000            16
356     LESTER      KRAUS     4.115000            16
290      JOANN    GARDNER     4.177500            16
35     ANTONIO       MEEK     4.927500            16
29         ANN      EVANS     4.519412            17
164     DWIGHT   LOMBARDI     4.401765            17
408    MELINDA  FERNANDEZ     4.754706            17
329    KENNETH     GOODEN     4.990000            17
200      FLOYD      GANDY     4.107647            17
571     VERNON      CHAPA     4.934444            18
364     LONNIE     TIRADO     5.267778            18


-----------------------------------
82. HANGİ MAĞAZALARDA HANGİ KATEGORİLER EN ÇOK KİRALANMIŞ?
-----------------------------------

In [249]:
df = {}

for table_name in ["store", "staff", "rental", "inventory", "film", "film_category", "category"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

store_categories = (
    df["store"]
      .merge(df["staff"], on="store_id", how="inner", suffixes=("", "_staff"))
      .merge(df["rental"], on="staff_id", how="inner", suffixes=("", "_rental"))
      .merge(df["inventory"], on="inventory_id", how="inner", suffixes=("", "_inv"))
      .merge(df["film"], on="film_id", how="inner", suffixes=("", "_film"))
      .merge(df["film_category"], on="film_id", how="inner", suffixes=("", "_fc"))
      .merge(df["category"], on="category_id", how="inner", suffixes=("", "_cat"))
      .groupby(["store_id", "name"], as_index=False)
      .agg(rental_count=("rental_id", "count"))
      .rename(columns={"name": "genre"})
      .sort_values(by="rental_count", ascending=False)
)

print(store_categories)

    store_id        genre  rental_count
30         2       Sports           614
17         2    Animation           584
1          1    Animation           582
7          1       Family           565
14         1       Sports           565
0          1       Action           562
13         1       Sci-Fi           553
16         2       Action           550
29         2       Sci-Fi           548
24         2      Foreign           543
22         2        Drama           532
23         2       Family           531
6          1        Drama           528
5          1  Documentary           525
21         2  Documentary           525
9          1        Games           494
8          1      Foreign           490
12         1          New           489
2          1     Children           483
25         2        Games           475
4          1       Comedy           475
3          1     Classics           475
20         2       Comedy           466
19         2     Classics           464


-----------------------------------
83. HANGİ MAĞAZALARDA HANGİ KATEGORİLER EN AZ KİRALANMIŞ?
-----------------------------------

In [250]:
df = {}

for table_name in ["store", "staff", "rental", "inventory", "film", "film_category", "category"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

store_categories = (
    df["store"]
      .merge(df["staff"], on="store_id", how="inner", suffixes=("", "_staff"))
      .merge(df["rental"], on="staff_id", how="inner", suffixes=("", "_rental"))
      .merge(df["inventory"], on="inventory_id", how="inner", suffixes=("", "_inv"))
      .merge(df["film"], on="film_id", how="inner", suffixes=("", "_film"))
      .merge(df["film_category"], on="film_id", how="inner", suffixes=("", "_fc"))
      .merge(df["category"], on="category_id", how="inner", suffixes=("", "_cat"))
      .groupby(["store_id", "name"], as_index=False)
      .agg(rental_count=("rental_id", "count"))
      .rename(columns={"name": "genre"})
      .sort_values(by="rental_count", ascending=True)
)

print(store_categories)

    store_id        genre  rental_count
27         2        Music           398
15         1       Travel           399
26         2       Horror           423
10         1       Horror           423
11         1        Music           432
31         2       Travel           438
28         2          New           451
18         2     Children           462
19         2     Classics           464
20         2       Comedy           466
4          1       Comedy           475
3          1     Classics           475
25         2        Games           475
2          1     Children           483
12         1          New           489
8          1      Foreign           490
9          1        Games           494
21         2  Documentary           525
5          1  Documentary           525
6          1        Drama           528
23         2       Family           531
22         2        Drama           532
24         2      Foreign           543
29         2       Sci-Fi           548


-----------------------------------
84. EN FAZLA SAYIDA FİLM KİRALAYAN MAĞAZA HANGİSİDİR VE TOPLAM KİRALAMA SAYISI NEDİR?
-----------------------------------

In [251]:
df = {}

for table_name in ["store", "staff", "rental"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

top_rental_store = (
    df["store"]
      .merge(df["staff"], on="store_id")
      .merge(df["rental"], on="staff_id")
      .groupby("store_id", as_index=False)
      .agg(total_rentals=("rental_id", "count"))
      .sort_values(by="total_rentals", ascending=False)
      .head(1)
)

print(top_rental_store)

Store with the highest number of rentals:
   store_id  total_rentals
0         1           8040


-----------------------------------
85. EN AZ SAYIDA FİLM KİRALAYAN MAĞAZA HANGİSİDİR VE TOPLAM KİRALAMA SAYISI NEDİR?
-----------------------------------

In [253]:
df = {}

for table_name in ["store", "staff", "rental"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

top_rental_store = (
    df["store"]
      .merge(df["staff"], on="store_id")
      .merge(df["rental"], on="staff_id")
      .groupby("store_id", as_index=False)
      .agg(total_rentals=("rental_id", "count"))
      .sort_values(by="total_rentals", ascending=False)
      .head(1)
)

print(top_rental_store)

   store_id  total_rentals
0         1           8040


-----------------------------------
86. EN FAZLA SAYIDA KİRALAMA YAPAN MÜŞTERİ HANGİSİDİR VE TOPLAM KİRALAMA SAYISI NEDİR?
-----------------------------------

In [254]:
df = {}

for table_name in ["store", "staff", "rental", "customer"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

top_renter = (
    df["customer"]
      .merge(df["rental"], on="customer_id")
      .groupby(["first_name", "last_name"], as_index=False)
      .agg(total_rentals=("rental_id", "count"))
      .sort_values(by="total_rentals", ascending=False)
      .head(1)
)

print(top_renter)

    first_name last_name  total_rentals
175    ELEANOR      HUNT             46


-----------------------------------
87. EN AZ SAYIDA KİRALAMA YAPAN MÜŞTERİ HANGİSİDİR VE TOPLAM KİRALAMA SAYISI NEDİR?
-----------------------------------

In [255]:
df = {}
for table_name in ["store", "staff", "rental", "customer"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

# Customer with the highest rental count
top_renter = (
    df["customer"]
      .merge(df["rental"], on="customer_id")
      .groupby(["first_name", "last_name"], as_index=False)
      .agg(total_rentals=("rental_id", "count"))
      .sort_values(by="total_rentals", ascending=True)
      .head(1)
)

print("Customer with the highest number of rentals:")
print(top_renter)

Customer with the highest number of rentals:
   first_name last_name  total_rentals
71      BRIAN     WYMAN             12


-----------------------------------
88. EN FAZLA SAYIDA KİRALANAN FİLM HANGİSİDİR VE TOPLAM KİRALANMA SAYISI NEDİR?
-----------------------------------

In [256]:
df = {}

for table_name in ["rental", "film", "inventory"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

most_rented_film = (
    df["film"]
      .merge(df["inventory"], on="film_id")
      .merge(df["rental"], on="inventory_id")
      .groupby("title", as_index=False)
      .agg(total_rentals=("rental_id", "count"))
      .sort_values(by="total_rentals", ascending=False)
      .head(1)
)

print(most_rented_film)

                 title  total_rentals
96  BUCKET BROTHERHOOD             34


-----------------------------------
89. EN AZ SAYIDA KİRALANAN FİLM HANGİSİDİR VE TOPLAM KİRALANMA SAYISI NEDİR?
-----------------------------------

In [257]:
df = {}

for table_name in ["rental", "film", "inventory"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

most_rented_film = (
    df["film"]
      .merge(df["inventory"], on="film_id")
      .merge(df["rental"], on="inventory_id")
      .groupby("title", as_index=False)
      .agg(total_rentals=("rental_id", "count"))
      .sort_values(by="total_rentals", ascending=True)
      .head(1)
)

print(most_rented_film)

           title  total_rentals
558  MIXED DOORS              4


-----------------------------------
90. EN FAZLA KAZANÇ SAĞLAYAN MÜŞTERİ HANGİSİDİR VE TOPLAM KAZANCI NEDİR?
-----------------------------------

In [258]:
df = {}

for table_name in ["rental", "customer", "payment"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

max_revenue_customer = (
    df["customer"]
      .merge(df["rental"], on="customer_id")
      .merge(df["payment"], on="rental_id")
      .groupby(["first_name", "last_name"], as_index=False)
      .agg(total_revenue=("amount", "sum"))
      .sort_values(by="total_revenue", ascending=False)
      .head(1)
)

print(max_revenue_customer)

    first_name last_name  total_revenue
318       KARL      SEAL         221.55


-----------------------------------
91. EN AZ KAZANÇ SAĞLAYAN MÜŞTERİ HANGİSİDİR VE TOPLAM KAZANCI NEDİR?
-----------------------------------

In [259]:
df = {}
for table_name in ["rental", "customer", "payment"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

# Customer with the highest total revenue
max_revenue_customer = (
    df["customer"]
      .merge(df["rental"], on="customer_id")
      .merge(df["payment"], on="rental_id")
      .groupby(["first_name", "last_name"], as_index=False)
      .agg(total_revenue=("amount", "sum"))
      .sort_values(by="total_revenue", ascending=True)
      .head(1)
)

print("Customer with the highest revenue and their total amount:")
print(max_revenue_customer)

Customer with the highest revenue and their total amount:
   first_name last_name  total_revenue
83   CAROLINE    BOWMAN          50.85


-----------------------------------
92. HANGİ FİLM, HANGİ KATEGORİDE EN ÇOK KAZANÇ SAĞLAMIŞ?
-----------------------------------

In [260]:
df = {}

for table_name in ["film", "film_category", "category", "inventory", "rental", "payment"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

max_revenue_film_category = (
    df["film"]
      .merge(df["film_category"], on="film_id", suffixes=("", "_fc"))
      .merge(df["category"], on="category_id", suffixes=("", "_cat"))
      .merge(df["inventory"], on="film_id")
      .merge(df["rental"], on="inventory_id")
      .merge(df["payment"], on="rental_id", suffixes=("", "_pay"))
      .groupby(["title", "name"])
      .agg(total_revenue=("amount", "sum"))
      .reset_index()
      .sort_values(by="total_revenue", ascending=False)
      .head(5)
)

print(max_revenue_film_category)

                 title         name  total_revenue
841   TELEGRAPH VOYAGE        Music         231.73
930          WIFE TURN  Documentary         223.69
957          ZORRO ARK       Comedy         214.69
348  GOODFELLAS SALUTE       Sci-Fi         209.69
730     SATURDAY LAMBS       Sports         204.72


-----------------------------------
93. HANGİ FİLM, HANGİ KATEGORİDE EN AZ KAZANÇ SAĞLAMIŞ?
-----------------------------------

In [261]:
df = {}
for table_name in ["film", "film_category", "category", "inventory", "rental", "payment"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

# Top 5 most profitable films and their categories
max_revenue_film_category = (
    df["film"]
      .merge(df["film_category"], on="film_id", suffixes=("", "_fc"))
      .merge(df["category"], on="category_id", suffixes=("", "_cat"))
      .merge(df["inventory"], on="film_id")
      .merge(df["rental"], on="inventory_id")
      .merge(df["payment"], on="rental_id", suffixes=("", "_pay"))
      .groupby(["title", "name"])
      .agg(total_revenue=("amount", "sum"))
      .reset_index()
      .sort_values(by="total_revenue", ascending=True)
      .head(5)
)

print("Top 5 most profitable films and their categories:")
print(max_revenue_film_category)

Top 5 most profitable films and their categories:
                 title         name  total_revenue
847        TEXAS WATCH       Horror           5.94
608   OKLAHOMA JUMANJI          New           5.94
315  FREEDOM CLEOPATRA       Comedy           5.95
244  DUFFEL APOCALYPSE  Documentary           6.93
953     YOUNG LANGUAGE  Documentary           6.93


-----------------------------------
94. EN FAZLA KAZANÇ SAĞLAYAN MAĞAZA HANGİSİDİR VE TOPLAM KAZANCI NEDİR?
-----------------------------------

In [262]:
df = {}

for table_name in ["store", "staff", "rental", "payment"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

max_revenue_store = (
    df["store"]
      .merge(df["staff"], on="store_id", suffixes=("", "_staff"))
      .merge(df["rental"], on="staff_id", suffixes=("", "_rental"))
      .merge(df["payment"], on="rental_id", suffixes=("", "_payment"))
      .groupby("store_id")
      .agg(total_revenue=("amount", "sum"))
      .reset_index()
      .sort_values(by="total_revenue", ascending=False)
      .head(1)
)

print(max_revenue_store)

   store_id  total_revenue
1         2       33881.94


-----------------------------------
95. EN AZ KAZANÇ SAĞLAYAN MAĞAZA HANGİSİDİR VE TOPLAM KAZANCI NEDİR?
-----------------------------------

In [263]:
df = {}

for table_name in ["store", "staff", "rental", "payment"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

max_revenue_store = (
    df["store"]
      .merge(df["staff"], on="store_id", suffixes=("", "_staff"))
      .merge(df["rental"], on="staff_id", suffixes=("", "_rental"))
      .merge(df["payment"], on="rental_id", suffixes=("", "_payment"))
      .groupby("store_id")
      .agg(total_revenue=("amount", "sum"))
      .reset_index()
      .sort_values(by="total_revenue", ascending=True)
      .head(1)
)

print(max_revenue_store)

   store_id  total_revenue
0         1       33524.62


-----------------------------------
96. EN FAZLA SAYIDA FARKLI FİLM KİRALAYAN MÜŞTERİ HANGİSİDİR?
-----------------------------------

In [264]:
df = {}

for table_name in ["rental", "customer"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

unique_rentals = (
    df["customer"]
      .merge(df["rental"], on="customer_id")
      .groupby(["first_name", "last_name"])
      .agg(unique_rental_count=("inventory_id", "nunique"))
      .reset_index()
      .sort_values(by="unique_rental_count", ascending=False)
      .head(1)
)

print(unique_rentals)

    first_name last_name  unique_rental_count
175    ELEANOR      HUNT                   46


-----------------------------------
97. EN AZ SAYIDA FARKLI FİLM KİRALAYAN MÜŞTERİ HANGİSİDİR?
-----------------------------------

In [265]:
df = {}

for table_name in ["rental", "customer"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

unique_rentals = (
    df["customer"]
      .merge(df["rental"], on="customer_id")
      .groupby(["first_name", "last_name"])
      .agg(unique_rental_count=("inventory_id", "nunique"))
      .reset_index()
      .sort_values(by="unique_rental_count", ascending=True)
      .head(1)
)

print(unique_rentals)

   first_name last_name  unique_rental_count
71      BRIAN     WYMAN                   12


-----------------------------------
98. EN FAZLA SAYIDA FARKLI MÜŞTERİ TARAFINDAN KİRALANAN FİLM HANGİSİDİR?
-----------------------------------

In [266]:
df = {}

for table_name in ["rental", "film", "inventory"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

unique_customer_film = (
    df["film"]
      .merge(df["inventory"], on="film_id")
      .merge(df["rental"], on="inventory_id")
      .groupby("title")
      .agg(unique_customer_count=("customer_id", "nunique"))
      .reset_index()
      .sort_values(by="unique_customer_count", ascending=False)
      .head(5)
)

print(unique_customer_film)

                   title  unique_customer_count
96    BUCKET BROTHERHOOD                     33
312       FORWARD TEMPLE                     32
361       GRIT CLOCKWORK                     32
697  RIDGEMONT SUBMARINE                     32
733        SCALAWAG DUCK                     32


-----------------------------------
99. EN AZ SAYIDA FARKLI MÜŞTERİ TARAFINDAN KİRALANAN FİLM HANGİSİDİR?
-----------------------------------

In [267]:
df = {}
for table_name in ["rental", "film", "inventory"]:
    cur = conn.execute(f"SELECT * FROM {table_name}")
    cols = [c[0] for c in cur.description]
    df[table_name] = pd.DataFrame(cur.fetchall(), columns=cols)

# Films rented by the highest number of unique customers
unique_customer_film = (
    df["film"]
      .merge(df["inventory"], on="film_id")
      .merge(df["rental"], on="inventory_id")
      .groupby("title")
      .agg(unique_customer_count=("customer_id", "nunique"))
      .reset_index()
      .sort_values(by="unique_customer_count", ascending=True)
      .head(5)
)

print("Films rented by the highest number of different customers:")
print(unique_customer_film)

Films rented by the highest number of different customers:
              title  unique_customer_count
866     TRAIN BUNCH                      4
417    HUNTER ALTER                      4
558     MIXED DOORS                      4
378  HARDLY ROBBERS                      4
747     SEVEN SWARM                      5


-----------------------------------
100. HANGİ MAĞAZADA, HANGİ KATEGORİDE EN FAZLA KAZANÇ SAĞLANMIŞ?
-----------------------------------

In [271]:
df = {}

for t in ["store", "staff", "rental", "payment", "inventory", "film", "film_category", "category"]:
    cur = conn.execute(f"SELECT * FROM {t}")
    df[t] = pd.DataFrame(cur.fetchall(), columns=[c[0] for c in cur.description])

rent_pay = (
    df["rental"][["rental_id", "staff_id", "inventory_id"]]
      .merge(df["payment"][["rental_id", "amount"]], on="rental_id", how="inner")
)

inv_genre = (
    df["inventory"][["inventory_id", "film_id"]]
      .merge(df["film"][["film_id"]], on="film_id", how="inner")
      .merge(df["film_category"][["film_id", "category_id"]], on="film_id", how="inner")
      .merge(df["category"][["category_id", "name"]], on="category_id", how="inner")
)

staff_store = (
    df["staff"][["staff_id", "store_id"]]
      .merge(df["store"][["store_id"]], on="store_id", how="inner")
)

# 4) Put all together and aggregate
store_category_revenue = (
    rent_pay
      .merge(inv_genre,  on="inventory_id", how="inner")
      .merge(staff_store, on="staff_id",    how="inner")
      .groupby(["store_id", "name"], as_index=False)
      .agg(total_revenue=("amount", "sum"))
      .sort_values("total_revenue", ascending=False)
      .head(1)
)

print("Store and category with the highest revenue:")
print(store_category_revenue)

Store and category with the highest revenue:
    store_id    name  total_revenue
30         2  Sports        2761.85
